In [1]:
import os
import pandas as pd
import numpy as np
from snowflake.sqlalchemy import URL
import logging
import warnings
from pathlib import Path
import datetime as datetime
from dateutil.relativedelta import relativedelta

warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.INFO)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 1000)
logging.basicConfig(level="INFO")
logger = logging.getLogger()
root = Path().cwd()

# add all the required files in this path
input_path = root / "inputs"
if not os.path.exists(input_path):
    os.makedirs(input_path, exist_ok=True)

# save all the files in this path
output_path = root / "outputs"
if not os.path.exists(output_path):
    os.makedirs(output_path, exist_ok=True)
logger.info(f"Reading dependencies from {input_path}...")
logger.info(f"saving files to {root}...")

/Users/keyurpethad/anaconda3/lib/python3.11/site-packages/snowflake/connector/options.py:103: UserWarning: You have an incompatible version of 'pyarrow' installed (11.0.0), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(
INFO:root:Reading dependencies from /Users/keyurpethad/Library/CloudStorage/GoogleDrive-keyur@niro.money/My Drive/analytics-projects/Portfolio Analysis/Analysis/inputs...
INFO:root:saving files to /Users/keyurpethad/Library/CloudStorage/GoogleDrive-keyur@niro.money/My Drive/analytics-projects/Portfolio Analysis/Analysis...


In [2]:
curr_month = "Jan'24"
prev_month = "Dec'23"

In [3]:
from sqlalchemy import create_engine


class extractorFromQuery:
    def __init__(self):
        self.credentials = {
            "account": "qfi-niro",
            "user": "keyur",
            "password": "Keyur@17061996",
            "warehouse": "NIRO_ANALYTICS_WAREHOUSE",
        }

    def queryExecutor(self, query: str, *args):
        engine = create_engine(URL(**self.credentials))
        res = pd.read_sql_query(query, engine, params=args)
        return res

In [4]:
disbursed_cases = """
    select 
    la.id,
    ou.category,
    la.user_id,
    la.principal_amount,
    la.interest_rate,
    la.tenure,
    la.selfie_liveness_score,
    la.selfie_liveness_result,
    la.kyc_status,
    la.pan_retry_count,
    la.nach_retry_count,
    la.remote_cpv_status,
    la.ckyc_status,
    o.attributed_partner as "supply",
    o.propensity,
    o.offer_type,
    o.phone_number,
    o.hashed_phone,
    tu.customer_availability,
    tu.income_source,
    tu.purpose_of_loan,
    tu.monthly_take_home_income,
    tu.employer_name,
    tu.nature_of_business,
    tu.cpv_need_to_initiate_at,
    tu.residence_ownership_type,
    tu.marital_status,
    tu.designation,
    cpv.cpv_type,
    cpv.agency_status,
    l.name as "demand",
    ba.bank_name,
    CASE
        when la.loan_disbursed_date_by_ops is null then date(la.disbursement_date)
        else date(la.loan_disbursed_date_by_ops)
    end as disbursement_date
    from niro_rds.niro_data.core__public__loan_application la
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__OFFERS o on o.id = la.offer_id
    left join NIRO_RDS.NIRO_DATA.core__public__offer_user_details ou on ou.offer_id = o.id
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__VERIFICATION_DETAILS vd on vd.id = la.verification_id 
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__TELE_PD_USER tu on vd.telepd_id::bigint = tu.id
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__CP_VERIFICATION cpv on cpv.id = vd.cpv_id::bigint
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__LENDERS l on l.id = o.lender_uid
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__BANK_ACCOUNTS b on b.id = la.bank_account_id
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__BANK_BRANCHES ba on b.ifsc_code = ba.ifsc_code    
    where la.loan_application_status in ('CLOSED','LOAN_EXECUTED') and la.is_deleted = False and o.is_deleted = False;
"""
total_disbursed_cases = extractorFromQuery().queryExecutor(disbursed_cases)
total_disbursed_cases.shape

INFO:snowflake.connector.connection:Snowflake Connector for Python Version: 3.4.0, Python Version: 3.11.5, Platform: macOS-10.16-x86_64-i386-64bit
INFO:snowflake.connector.connection:This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
INFO:snowflake.connector.cursor:query: [select current_database(), current_schema();]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [ROLLBACK]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [select la.id, ou.category, la.user_id, la.principal_amount, la.interest_rate, la...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connect

(45626, 33)

In [5]:
total_disbursed_cases.bank_name.value_counts()

bank_name
HDFC BANK                                      9014
STATE BANK OF INDIA                            8884
ICICI BANK LIMITED                             4394
KOTAK MAHINDRA BANK LIMITED                    4009
AXIS BANK                                      3350
CANARA BANK                                    2273
UNION BANK OF INDIA                            2230
BANK OF BARODA                                 2189
PUNJAB NATIONAL BANK                           1566
INDIAN BANK                                    1256
BANK OF INDIA                                  1016
FEDERAL BANK                                    754
INDUSIND BANK                                   616
IDBI BANK                                       606
KARNATAKA BANK LIMITED                          524
BANK OF MAHARASHTRA                             429
IDFC FIRST BANK LTD                             418
YES BANK                                        285
CITY UNION BANK LIMITED                         219
IN

In [6]:
#Dropped following field from above due to stated reasons

# la.selfie_match_result, -- large no. of missing data
# la.selfie_match_confidence, -- large no. of missing data
# la.loan_acceptance_status, -- all values "COMPLETED"
# la.nach_status, -- all values "COMPLETED"
# la.loan_agreement_status, -- all values "COMPLETED"
# la.disbursement_status, -- all values "COMPLETED"
# la.selfie_status, -- all values "COMPLETED"
# la.bank_account_status, -- all values "COMPLETED"
# la.kyc_retry_count, -- all values "COMPLETED"
# la.bank_retry_count, -- all values "COMPLETED"
# la.selfie_retry_count, -- all values "COMPLETED"
# la.redo_kyc, -- all values "FALSE"
# la.disbursement_remarks, -- irrelevant
# la.documents_pending, -- all values "FALSE"
# la.documents_remarks, -- irrelevant
# la.name_match_score, -- all values (blank)
# la.employment_verification_status, -- all values (blank)
# la.monthly_income_verification_status, -- all values (blank)
# la.lender_policy_check_status, -- all values (blank)
# la.document_verification_status, -- all values (blank)
# ou.propensity_score, -- all values (blank)
# ou.existing_obligation, -- all values (blank)
# tu.gender, -- less missing values in performance data
# tu.no_of_years_in_employment, -- data unclean (needs to be a dropdown)
# tu.final_telepd_status, -- all values "COMPLETED"
# tu.rent_amount, -- unclean data
# cpv.cpv_status, -- unclean data
# o.data_tag, -- not sure of use

# od.dec_reason, -- all values "NOT DECLINED"


In [7]:
total_disbursed_cases.head()

,id,category,user_id,principal_amount,interest_rate,tenure,selfie_liveness_score,selfie_liveness_result,kyc_status,pan_retry_count,nach_retry_count,remote_cpv_status,ckyc_status,supply,propensity,offer_type,phone_number,hashed_phone,customer_availability,income_source,purpose_of_loan,monthly_take_home_income,employer_name,nature_of_business,cpv_need_to_initiate_at,residence_ownership_type,marital_status,designation,cpv_type,agency_status,demand,bank_name,disbursement_date
0,8ff8e963-62be-4a29-9a95-727af5659b7a,CAT-C,3258321a-628c-430d-a223-0593c79bd79e,66701.0,26.9,18.0,0.99,PASS,PENDING,0.0,0.0,COMPLETED,COMPLETED,quikr,HiiProp,PRE_APPROVED,9589055084,41f8a882471ed3fbc2657746691e0badecf5f39e8d7f73...,Yes,Self Employed,Health/Hospital,23000,-,service,Residence,Family owned,MARRIED,-,REMOTE,Positive,liquiloans,BANK OF BARODA,2023-03-27
1,e79c178d-181c-43fb-b86c-413110addb90,CAT-A,a075e629-1f07-4218-a949-820a8ca0ccfd,267903.0,16.8,36.0,0.36,FAIL,COMPLETED,1.0,0.0,COMPLETED,PENDING,niro,LowProp,PRE_APPROVED,9221348191,061c33255a6e18d08fd3ab09734ad30474a853be0c8732...,Yes,Salaried,Purchase,35000,ICMS,-,Residence,Self owned,MARRIED,senior technician,REMOTE,Positive,payu,HDFC BANK,2023-05-11
2,addb6c7e-977e-4baa-9c72-f9954d77d0ef,CAT-D,bcf9b9be-931c-4b57-a348-32680cf85dfa,53068.0,23.9,12.0,1.00,PASS,COMPLETED,0.0,0.0,COMPLETED,PENDING,snapdeal,HiiProp,PRE_APPROVED,9965824404,6d4bdd34e810eee79a93d4a8e0731af8016a6602f1f00e...,Yes,Self Employed,Business Development,30000,-,FREE LANCER,Residence,Rented,SINGLE,-,PHYSICAL,Positive,payu,CANARA BANK,2023-04-13
3,09f52664-490a-4eae-8f72-df8faa207787,CAT-D,8a0e1a3d-d6ec-4a25-9e26-5b87c703e24a,68989.0,23.9,12.0,0.85,PASS,COMPLETED,0.0,0.0,COMPLETED,PENDING,snapdeal,HiiProp,PRE_APPROVED,8208550948,60e41cee77e642d4eeae2d6cf03d97caadf81473a586c3...,Yes,Salaried,Purchase,30500,K mart,-,Residence,Rented,MARRIED,Sales Executive,REMOTE,Positive,payu,UNION BANK OF INDIA,2023-03-20
4,3a77679d-2ea9-4a0a-886e-4a061e9caf3c,CAT-C,6ddda864-9f50-4b4f-ac7b-520e5fa2f2e6,106925.0,26.9,18.0,0.00,FAIL,PENDING,0.0,0.0,NOT_REQUIRED,COMPLETED,quikr,HiiProp,PRE_APPROVED,9993365606,fb8b3db3596071b6ed9f222556ea79b2d279d84659ecd8...,Yes,Self Employed,Purchase,100000,-,Real-estate,Residence,Family owned,SINGLE,-,None,Positive,liquiloans,HDFC BANK,2023-01-23


In [8]:
total_disbursed_cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45626 entries, 0 to 45625
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        45626 non-null  object 
 1   category                  43995 non-null  object 
 2   user_id                   45626 non-null  object 
 3   principal_amount          45626 non-null  float64
 4   interest_rate             45626 non-null  float64
 5   tenure                    45626 non-null  float64
 6   selfie_liveness_score     44009 non-null  float64
 7   selfie_liveness_result    44009 non-null  object 
 8   kyc_status                45626 non-null  object 
 9   pan_retry_count           45626 non-null  float64
 10  nach_retry_count          45623 non-null  float64
 11  remote_cpv_status         45605 non-null  object 
 12  ckyc_status               45626 non-null  object 
 13  supply                    45578 non-null  object 
 14  propen

In [9]:
disbursed_cases_with_cvattrs = """
            select * from (
            select 
            distinct la.id,
            pii.gender as cibil_gender,
            pii.pin as cibil_pincode,
            pii.calculated_age,
            pii.city as cibil_city,
            pii.state as cibil_state,
            pii.native_language,
            tli.*,
            date_trunc("MONTH",date(od.created_at)) as createdate,
            od.current_salary,
            od.available_income,
            od.foir_dlq, od.foir_cibil, od.foir_trd, od.foir_mean, od.new_foir, 
            od.naps_score,
            rank()over(partition by od.user_id order by od.created_at asc) as rnk,
            CASE
                when la.loan_disbursed_date_by_ops is null then date(la.disbursement_date)
                else date(la.loan_disbursed_date_by_ops)
            END as disbursement_date
            from NIRO_RDS.NIRO_DATA.CORE__PUBLIC__LOAN_APPLICATION la
            left join NIRO_RDS.NIRO_DATA.core__public__offer_user_details ou on ou.offer_id = la.offer_id
            left join ANALYTICS.OFFERS.offer_details od on od.user_id = la.user_id and od.category = ou.category
            left join ANALYTICS.OFFERS.personal_info pii on od.pii_id = pii.pii_id
            left join ANALYTICS.OFFERS.tradeline_info tli on tli.tli_id = od.tli_id
            where la.loan_application_status in ('CLOSED','LOAN_EXECUTED') 
            and la.is_deleted = False
            and(la.is_deleted_at_source = false or la.is_deleted_at_source is null)
            and od.dec_reason = 'NOT DECLINED')
where rnk = 1;
"""
disbursed_cases = (
    extractorFromQuery()
    .queryExecutor(disbursed_cases_with_cvattrs)
    .drop(columns="disbursement_date")
)
disbursed_cases.shape

INFO:snowflake.connector.connection:Snowflake Connector for Python Version: 3.4.0, Python Version: 3.11.5, Platform: macOS-10.16-x86_64-i386-64bit
INFO:snowflake.connector.connection:This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
INFO:snowflake.connector.cursor:query: [select current_database(), current_schema();]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [ROLLBACK]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [select * from ( select distinct la.id, pii.gender as cibil_gender, pii.pin as ci...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connect

(45044, 89)

In [10]:
disbursed_cases.head()

,id,cibil_gender,cibil_pincode,calculated_age,cibil_city,cibil_state,native_language,tli_id,pii_id,agg911,rvlr01,bcpmtstr,cv11,cv14,mt28s,mt33s,pl33s,at20s,mt01s,bc02s,bg01s,cv10,trd,at33a,au33s,co04s180,au28s,pl28s,co01s180,bc28s,cv12,co05s,g310s,aggs911,at01s,at33a_ne_ccod,cv14_12m,cv14_6m,cv14_3m,cv14_1m,g310s_24m,g310s_6m,g310s_3m,g310s_1m,cv11_24m,cv11_12m,g057s_1dpd_36m,g057s_1dpd_12m,bc106s_60dpd,bc107s_24m,bc106s_60dpd_12m,bc107s_12m,bc106s_le_30dpd_12m,bc09s_36m_hcsa_le_30,pl09s_36m_hcsa_le_30,at09s_6m,g310s_36m,at33a_ne_wo,at09s_12m,at09s_3m,cv13,cv24,revs904,cv20,cv22,ul_trd,cv21,g310s_2m,secured_accounts_count,unsecured_accounts_count,secured_high_credit_sum,unsecured_high_credit_sum,secured_amount_overdue_sum,unsecured_amount_overdue_sum,secured_balances_sum,unsecured_balances_sum,own_accounts_count,other_accounts_count,bureau_score,createdate,current_salary,available_income,foir_dlq,foir_cibil,foir_trd,foir_mean,new_foir,naps_score,rnk
0,985539cc-f0ff-4c39-aa21-f15d2cdeca12,Female,500018,21,HYDERABAD,ANDHRA PRADESH,English,fc329e5b-e2e1-48fd-8a41-393d466ccc97,b99d61b1-cfd9-497a-82d7-bcf80153b949,-2.00,-2.00,TRANSACTOR,0.0,7.0,-1.0,-1.0,9136.0,12.0,0.0,1.0,0.0,0.0,3.0,39028.0,-1.0,-1.0,-1.0,1000.0,0.0,29941.0,0.0,-1.0,1.5,-2.0,2.0,9136.0,5.0,5.0,5.0,3.0,1.5,1.0,1.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,1.0,1.5,-1.0,3.0,1.0,0.0,-2.0,29892.0,1494.60,29892.0,3.0,-2.0,0.0,0.0,3.0,0.0,36391.0,0.0,0.0,0.0,39028.0,0.0,3.0,712.0,2023-01-01,48000.0,26048.60,0.5,0.5,0.45,0.48,50.38,696.0,1
1,be9e6249-64fc-46da-a33a-a90ce01df0d0,Male,500062,52,HYDERABAD,ANDHRA PRADESH,English,6ba6be8c-ef9d-4471-9b76-bbfb827fe2fd,57d6d262-2da9-439a-a523-2f38b374a1f8,42.14,41.33,RVLRPLUS,0.0,35.0,1600000.0,1281716.0,569643.0,189.0,1.0,7.0,0.0,0.0,53.0,3054531.0,-1.0,-1.0,-1.0,863183.0,0.0,1090000.0,0.0,-1.0,1.0,42.0,42.0,2708224.0,4.0,3.0,3.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,53.0,53.0,0.0,0.0,0.0,0.0,10.0,0.0,3.0,4.0,1.0,-1.0,9.0,1.0,0.0,22568.0,63923.0,43022.35,346307.0,39.0,74043.0,1.0,11.0,23.0,5030627.0,1993025.0,0.0,0.0,2099704.0,954827.0,0.0,34.0,781.0,2023-08-01,457800.0,198328.77,0.5,0.4,0.36,0.42,25.48,900.0,1
2,451cd1a9-abd8-49f2-be3d-690908c6fe2b,Male,700039,38,KOLKATA,WEST BENGAL,Marathi,ab37c338-5c0c-45a9-90ac-e1f904a71d7d,8f8e5e6e-6bec-4d33-b5fc-ae5f1de02c60,60.25,0.00,INACTIVE,0.0,7.0,-1.0,-1.0,-3.0,26.0,0.0,1.0,0.0,0.0,4.0,2741.0,-1.0,-1.0,-1.0,-3.0,0.0,10000.0,0.0,-1.0,1.0,0.0,3.0,2741.0,3.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,-1.0,1.0,0.0,0.0,-2.0,4926.0,3315.00,0.0,4.0,4714.0,0.0,0.0,4.0,0.0,72499.0,0.0,0.0,0.0,2741.0,0.0,4.0,781.0,2022-12-01,48000.0,27862.95,0.5,0.4,0.50,0.47,45.84,726.0,1
3,c17de991-c2d5-48f7-9950-82bca2d0826d,Male,425409,28,JALGAON,MAHARASHTRA,None,f6a3c524-35d8-4830-a371-f8d4aa5189ae,cd4991fd-8985-4301-bc52-2113817613b6,16.95,0.00,TRANSACTOR,0.0,16.0,-1.0,-1.0,163719.0,41.0,0.0,1.0,0.0,0.0,15.0,184298.0,-1.0,-1.0,-1.0,280000.0,0.0,40000.0,0.0,-1.0,1.0,17.0,14.0,177519.0,5.0,4.0,2.0,0.0,1.5,1.0,1.0,0.0,0.0,0.0,15.0,15.0,0.0,0.0,0.0,0.0,1.0,1.0,4.0,3.0,1.5,-1.0,4.0,1.0,0.0,-2.0,4006.0,6363.95,6779.0,12.0,2697.0,1.0,3.0,12.0,599391.0,567779.0,0.0,0.0,0.0,184298.0,0.0,15.0,732.0,2023-10-01,48000.0,18785.10,0.5,0.5,0.45,0.48,78.79,808.0,1
4,5ef39193-1fed-4127-8787-ea70e23d8e22,Female,562149,26,CHIKKABALLAPUR,KARNATAKA,None,1d3094e4-167c-4c79-ae40-a0e7ec68e705,635fcb1e-aeb7-499a-9226-3685f80aeb35,-1.00,-1.00,NOBC,0.0,14.0,750000.0,734876.0,45914.0,19.0,1.0,-1.0,0.0,0.0,19.0,780790.0,-1.0,-1.0,-1.0,65000.0,0.0,-1.0,0.0,-1.0,1.0,-1.0,18.0,780790.0,12.0,3.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,19.0,19.0,-1.0,-1.0,-1.0,-1.0,-1.0,-5.0,16.0,2.0,1.0,-1.0,16.0,0.0,0.0,-1.0,-1.0,-2.00,-1.0,18.0,-2.0,0.0,1.0,18.0,750000.0,258863.0,0.0,0.0,734876.0,45914.0,0.0,19.0,757.0,2022-07-01,45000.0,17954.30,0.5,0.4,0.39,0.43,80.26,658.0,1


In [11]:
column_name_map = {'agg911': 'max_aggregate_bankcard_utilisation_l12m', 'rvlr01': 'cc_util_revolving_l1m', 'bcpmtstr': 'bank_card_payment_category', 'cv11': 'no_of_60p_accs_ever', 'cv14': 'no_of_deduped_inquiries', 
                   'mt28s': 'total_cl_open_mortgage_trades_l12m', 'mt33s': 'total_balance_open_mortgage_trades_l12m', 'pl33s':'total_bal_of_open_pl_l12m', 'at20s':'months_since_oldest_trade', 
                   'mt01s':'no_of_mortgage_trades', 'bc02s':'no_of_open_cc_trades', 'bg01s':'no_of_business_general_trades', 'cv10':'no_of_30p_accs_ever', 'trd':'no_of_trades', 'at33a':'total_bal_open_trades_l12m',
                   'au33s':'total_bal_open_auto_trades_l12m', 'co04s180':'months_since_recent_chargedoff180P', 'au28s':'total_cl_open_auto_trades_l12m', 'pl28s':'total_cl_open_pl_trades_l12m',
                   'co01s180':'no_of_chargedoff180P', 'bc28s':'total_cl_of_cc_trades_l12m', 'cv12':'no_of_90p_accs_ever', 'co05s':'total_bal_of_chargedoff_trades', 'g310s':'worst_rating_l12m',
                   'aggs911':'current_utilization_of_top_wallet_bankcard', 'at01s':'no_of_trades_pl_2w_gl_cd', 'at33a_ne_ccod':'total_bal_open_trades_l12m_except_wo_cc_od', 'cv14_12m':'no_of_deduped_inquiries_l12m',
                   'cv14_6m':'no_of_deduped_inquiries_l6m', 'cv14_3m':'no_of_deduped_inquiries_l3m', 'cv14_1m':'no_of_deduped_inquiries_l1m', 'g310s_24m':'worst_rating_l24m',
                   'g310s_6m':'worst_rating_l6m', 'g310s_3m':'worst_rating_l3m', 'g310s_1m':'worst_rating_l1m', 'cv11_24m':'no_of_60p_accs_l24m', 'cv11_12m':'no_of_60p_accs_l12m', 'g057s_1dpd_36m': 'no_of_trades_bounced_l36m',
                   'g057s_1dpd_12m':'no_of_trades_bounced_l12m', 'bc106s_60dpd':'no_of_cc_trades_60P_l24m', 'bc107s_24m':'no_of_30P_cc_l24m', 'bc106s_60dpd_12m':'no_of_cc_trades_60P_l12m', 'bc107s_12m':'no_of_30P_cc_l12m',
                   'bc106s_le_30dpd_12m':'no_of_cc_lt30p_l12m', 'bc09s_36m_hcsa_le_30':'no_cc_sanctionedamt_lt30K_l36m', 'pl09s_36m_hcsa_le_30':'no_pl_sanctionedamt_lt30K_l36m', 'at09s_6m':'no_tr_opened_l6m',
                   'g310s_36m':'worst_dpd_l36m', 'at33a_ne_wo':'total_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades', 'at09s_12m':'no_tr_opened_l12m', 'at09s_3m':'no_tr_opened_l3m', 'cv13':'per_of_30p_accs_ever',
                    'cv24':'total_payment_amount_bankcard_accs_l3m', 'revs904':'max_revolving_monthly_spend_l12m', 'cv20':'total_monthly_obligations_l3m', 'cv22':'total_balance_bankcard_accs_l3m', 
                    'ul_trd':'no_of_trades_unsec', 'cv21':'total_payment_amount_l3m', 'g310s_2m':'worst_dpd_l2m'}

In [12]:
disbursed_cases.rename(columns = column_name_map, inplace=True)

In [13]:
disbursed_cases.head()

,id,cibil_gender,cibil_pincode,calculated_age,cibil_city,cibil_state,native_language,tli_id,pii_id,max_aggregate_bankcard_utilisation_l12m,cc_util_revolving_l1m,bank_card_payment_category,no_of_60p_accs_ever,no_of_deduped_inquiries,total_cl_open_mortgage_trades_l12m,total_balance_open_mortgage_trades_l12m,total_bal_of_open_pl_l12m,months_since_oldest_trade,no_of_mortgage_trades,no_of_open_cc_trades,no_of_business_general_trades,no_of_30p_accs_ever,no_of_trades,total_bal_open_trades_l12m,total_bal_open_auto_trades_l12m,months_since_recent_chargedoff180P,total_cl_open_auto_trades_l12m,total_cl_open_pl_trades_l12m,no_of_chargedoff180P,total_cl_of_cc_trades_l12m,no_of_90p_accs_ever,total_bal_of_chargedoff_trades,worst_rating_l12m,current_utilization_of_top_wallet_bankcard,no_of_trades_pl_2w_gl_cd,total_bal_open_trades_l12m_except_wo_cc_od,no_of_deduped_inquiries_l12m,no_of_deduped_inquiries_l6m,no_of_deduped_inquiries_l3m,no_of_deduped_inquiries_l1m,worst_rating_l24m,worst_rating_l6m,worst_rating_l3m,worst_rating_l1m,no_of_60p_accs_l24m,no_of_60p_accs_l12m,no_of_trades_bounced_l36m,no_of_trades_bounced_l12m,no_of_cc_trades_60P_l24m,no_of_30P_cc_l24m,no_of_cc_trades_60P_l12m,no_of_30P_cc_l12m,no_of_cc_lt30p_l12m,no_cc_sanctionedamt_lt30K_l36m,no_pl_sanctionedamt_lt30K_l36m,no_tr_opened_l6m,worst_dpd_l36m,total_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades,no_tr_opened_l12m,no_tr_opened_l3m,per_of_30p_accs_ever,total_payment_amount_bankcard_accs_l3m,max_revolving_monthly_spend_l12m,total_monthly_obligations_l3m,total_balance_bankcard_accs_l3m,no_of_trades_unsec,total_payment_amount_l3m,worst_dpd_l2m,secured_accounts_count,unsecured_accounts_count,secured_high_credit_sum,unsecured_high_credit_sum,secured_amount_overdue_sum,unsecured_amount_overdue_sum,secured_balances_sum,unsecured_balances_sum,own_accounts_count,other_accounts_count,bureau_score,createdate,current_salary,available_income,foir_dlq,foir_cibil,foir_trd,foir_mean,new_foir,naps_score,rnk
0,985539cc-f0ff-4c39-aa21-f15d2cdeca12,Female,500018,21,HYDERABAD,ANDHRA PRADESH,English,fc329e5b-e2e1-48fd-8a41-393d466ccc97,b99d61b1-cfd9-497a-82d7-bcf80153b949,-2.00,-2.00,TRANSACTOR,0.0,7.0,-1.0,-1.0,9136.0,12.0,0.0,1.0,0.0,0.0,3.0,39028.0,-1.0,-1.0,-1.0,1000.0,0.0,29941.0,0.0,-1.0,1.5,-2.0,2.0,9136.0,5.0,5.0,5.0,3.0,1.5,1.0,1.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,1.0,1.5,-1.0,3.0,1.0,0.0,-2.0,29892.0,1494.60,29892.0,3.0,-2.0,0.0,0.0,3.0,0.0,36391.0,0.0,0.0,0.0,39028.0,0.0,3.0,712.0,2023-01-01,48000.0,26048.60,0.5,0.5,0.45,0.48,50.38,696.0,1
1,be9e6249-64fc-46da-a33a-a90ce01df0d0,Male,500062,52,HYDERABAD,ANDHRA PRADESH,English,6ba6be8c-ef9d-4471-9b76-bbfb827fe2fd,57d6d262-2da9-439a-a523-2f38b374a1f8,42.14,41.33,RVLRPLUS,0.0,35.0,1600000.0,1281716.0,569643.0,189.0,1.0,7.0,0.0,0.0,53.0,3054531.0,-1.0,-1.0,-1.0,863183.0,0.0,1090000.0,0.0,-1.0,1.0,42.0,42.0,2708224.0,4.0,3.0,3.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,53.0,53.0,0.0,0.0,0.0,0.0,10.0,0.0,3.0,4.0,1.0,-1.0,9.0,1.0,0.0,22568.0,63923.0,43022.35,346307.0,39.0,74043.0,1.0,11.0,23.0,5030627.0,1993025.0,0.0,0.0,2099704.0,954827.0,0.0,34.0,781.0,2023-08-01,457800.0,198328.77,0.5,0.4,0.36,0.42,25.48,900.0,1
2,451cd1a9-abd8-49f2-be3d-690908c6fe2b,Male,700039,38,KOLKATA,WEST BENGAL,Marathi,ab37c338-5c0c-45a9-90ac-e1f904a71d7d,8f8e5e6e-6bec-4d33-b5fc-ae5f1de02c60,60.25,0.00,INACTIVE,0.0,7.0,-1.0,-1.0,-3.0,26.0,0.0,1.0,0.0,0.0,4.0,2741.0,-1.0,-1.0,-1.0,-3.0,0.0,10000.0,0.0,-1.0,1.0,0.0,3.0,2741.0,3.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,-1.0,1.0,0.0,0.0,-2.0,4926.0,3315.00,0.0,4.0,4714.0,0.0,0.0,4.0,0.0,72499.0,0.0,0.0,0.0,2741.0,0.0,4.0,781.0,2022-12-01,48000.0,27862.95,0.5,0.4,0.50,0.47,45.84,726.0,1
3,c17de991-c2d5-48f7-9950-82bca2d0826d,Male,425409,28,JALGAON,MAHARASHTRA,None,f6a3c524-35d8-4830-a371-f8d4aa5189ae,cd4991fd-8985-4301-bc52-2113817613b6,16.95,0.00,TRANSACTOR,0.0,16.0,-1.0,-1.0,163719.0,41.0,0.0,1.0,0.0,0.0,15.0,184298.0,-1.0,-1.0,-1.0,280000.0,0.0,40000.0,0.0,-1.0,1.

In [14]:
disbursed_cases = disbursed_cases.drop_duplicates(subset=["id"])
final = total_disbursed_cases.merge(disbursed_cases, on="id", how="left").rename(
    columns={"id": "niro_opportunity_id"}
)
final.drop_duplicates(subset=["niro_opportunity_id"], inplace=True)
final.shape

(45626, 121)

In [15]:
final.head()

,niro_opportunity_id,category,user_id,principal_amount,interest_rate,tenure,selfie_liveness_score,selfie_liveness_result,kyc_status,pan_retry_count,nach_retry_count,remote_cpv_status,ckyc_status,supply,propensity,offer_type,phone_number,hashed_phone,customer_availability,income_source,purpose_of_loan,monthly_take_home_income,employer_name,nature_of_business,cpv_need_to_initiate_at,residence_ownership_type,marital_status,designation,cpv_type,agency_status,demand,bank_name,disbursement_date,cibil_gender,cibil_pincode,calculated_age,cibil_city,cibil_state,native_language,tli_id,pii_id,max_aggregate_bankcard_utilisation_l12m,cc_util_revolving_l1m,bank_card_payment_category,no_of_60p_accs_ever,no_of_deduped_inquiries,total_cl_open_mortgage_trades_l12m,total_balance_open_mortgage_trades_l12m,total_bal_of_open_pl_l12m,months_since_oldest_trade,no_of_mortgage_trades,no_of_open_cc_trades,no_of_business_general_trades,no_of_30p_accs_ever,no_of_trades,total_bal_open_trades_l12m,total_bal_open_auto_trades_l12m,months_since_recent_chargedoff180P,total_cl_open_auto_trades_l12m,total_cl_open_pl_trades_l12m,no_of_chargedoff180P,total_cl_of_cc_trades_l12m,no_of_90p_accs_ever,total_bal_of_chargedoff_trades,worst_rating_l12m,current_utilization_of_top_wallet_bankcard,no_of_trades_pl_2w_gl_cd,total_bal_open_trades_l12m_except_wo_cc_od,no_of_deduped_inquiries_l12m,no_of_deduped_inquiries_l6m,no_of_deduped_inquiries_l3m,no_of_deduped_inquiries_l1m,worst_rating_l24m,worst_rating_l6m,worst_rating_l3m,worst_rating_l1m,no_of_60p_accs_l24m,no_of_60p_accs_l12m,no_of_trades_bounced_l36m,no_of_trades_bounced_l12m,no_of_cc_trades_60P_l24m,no_of_30P_cc_l24m,no_of_cc_trades_60P_l12m,no_of_30P_cc_l12m,no_of_cc_lt30p_l12m,no_cc_sanctionedamt_lt30K_l36m,no_pl_sanctionedamt_lt30K_l36m,no_tr_opened_l6m,worst_dpd_l36m,total_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades,no_tr_opened_l12m,no_tr_opened_l3m,per_of_30p_accs_ever,total_payment_amount_bankcard_accs_l3m,max_revolving_monthly_spend_l12m,total_monthly_obligations_l3m,total_balance_bankcard_accs_l3m,no_of_trades_unsec,total_payment_amount_l3m,worst_dpd_l2m,secured_accounts_count,unsecured_accounts_count,secured_high_credit_sum,unsecured_high_credit_sum,secured_amount_overdue_sum,unsecured_amount_overdue_sum,secured_balances_sum,unsecured_balances_sum,own_accounts_count,other_accounts_count,bureau_score,createdate,current_salary,available_income,foir_dlq,foir_cibil,foir_trd,foir_mean,new_foir,naps_score,rnk
0,8ff8e963-62be-4a29-9a95-727af5659b7a,CAT-C,3258321a-628c-430d-a223-0593c79bd79e,66701.0,26.9,18.0,0.99,PASS,PENDING,0.0,0.0,COMPLETED,COMPLETED,quikr,HiiProp,PRE_APPROVED,9589055084,41f8a882471ed3fbc2657746691e0badecf5f39e8d7f73...,Yes,Self Employed,Health/Hospital,23000,-,service,Residence,Family owned,MARRIED,-,REMOTE,Positive,liquiloans,BANK OF BARODA,2023-03-27,Female,110058,44.0,DELHI,DELHI,Marathi,9cca07c1-86bc-4960-9c7d-e92cb839223f,3bdaa557-9b6a-48aa-8422-ba31c092d1c3,-2.0,-2.0,NOBC,0.0,7.0,-1.0,-1.0,-3.0,184.0,0.0,0.0,0.0,0.0,3.0,178059.0,-1.0,-1.0,-1.0,-3.0,0.0,-3.0,0.0,-1.0,1.0,-2.0,1.0,178059.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,3.0,3.0,0.0,-2.0,0.0,-2.0,1.0,0.0,0.0,0.0,1.0,-1.0,1.0,0.0,0.0,-7.0,-2.0,-2.0,-7.0,3.0,6921.0,0.0,0.0,1.0,0.0,200000.0,0.0,0.0,0.0,178059.0,0.0,1.0,738.0,2022-12-01,48000.00,19097.05,0.5,0.5,0.50,0.50,76.76,744.0,1.0
1,e79c178d-181c-43fb-b86c-413110addb90,CAT-A,a075e629-1f07-4218-a949-820a8ca0ccfd,267903.0,16.8,36.0,0.36,FAIL,COMPLETED,1.0,0.0,COMPLETED,PENDING,niro,LowProp,PRE_APPROVED,9221348191,061c33255a6e18d08fd3ab09734ad30474a853be0c8732...,Yes,Salaried,Purchase,35000,ICMS,-,Residence,Self owned,MARRIED,senior technician,REMOTE,Positive,payu,HDFC BANK,2023-05-11,Male,400605,42.0,MUMBAI,MAHARASHTRA,Kannada,73211792-22e2-4de0-8a9c-337a2a349dd0,d66a3c4f-4f3b-406c-8d07-884c7df72dd0,0.0,0.0,TRANSACTOR,0.0,13.0,-1.0,-1.0,-1.0,140.0,0.0,3.0,0.0,0.0,10.0,1187795.0,1156827.0,-1.0,1189179.0,-1.0,0.0,241778.0,0.0,-1.0,1.0,-2.0,1.0,1156827.0,2.0,2.0,1.0,0.0,

In [16]:
cols = [
    "secured_accounts_count",
    "unsecured_accounts_count",
    "secured_high_credit_sum",
    "unsecured_high_credit_sum",
    "secured_amount_overdue_sum",
    "unsecured_amount_overdue_sum",
    "secured_balances_sum",
    "unsecured_balances_sum",
]

normal_cols = [
    "user_id",
    "niro_opportunity_id",
    "pin",
    "city",
    "income_source",
    "monthly_take_home_income",
    "current_salary",
    "cpv_type",
    "principal_amount",
    "interest_rate",
    "tenure",
    "supply",
    "foir_dlq",
    "foir_cibil",
    "foir_trd",
    "foir_mean",
    "new_foir",
    "demand",
    "disbursement_date",
    "createdate",
    "dec_reason",
    "naps_score",
    "phone_number",
    "bureau_score",
    "calculated_age",
    "own_accounts_count",
    "other_accounts_count",
]

final.rename(
    columns={i: i.replace("_", " ").title() for i in cols}, inplace=True, errors="raise"
)
final.rename(
    columns={
        "bureau_score": "CIBILTUSC3 Score Value",
        "calculated_age": "Calculated_Age",
        "new_foir": "New_FOIR",
        "current_salary": "Current_Salary",
        "other_accounts_count": "Other Accounts count",
        "own_accounts_count": "Own Accounts count",
    },
    inplace=True,
    errors="raise",
)
final.head()

,niro_opportunity_id,category,user_id,principal_amount,interest_rate,tenure,selfie_liveness_score,selfie_liveness_result,kyc_status,pan_retry_count,nach_retry_count,remote_cpv_status,ckyc_status,supply,propensity,offer_type,phone_number,hashed_phone,customer_availability,income_source,purpose_of_loan,monthly_take_home_income,employer_name,nature_of_business,cpv_need_to_initiate_at,residence_ownership_type,marital_status,designation,cpv_type,agency_status,demand,bank_name,disbursement_date,cibil_gender,cibil_pincode,Calculated_Age,cibil_city,cibil_state,native_language,tli_id,pii_id,max_aggregate_bankcard_utilisation_l12m,cc_util_revolving_l1m,bank_card_payment_category,no_of_60p_accs_ever,no_of_deduped_inquiries,total_cl_open_mortgage_trades_l12m,total_balance_open_mortgage_trades_l12m,total_bal_of_open_pl_l12m,months_since_oldest_trade,no_of_mortgage_trades,no_of_open_cc_trades,no_of_business_general_trades,no_of_30p_accs_ever,no_of_trades,total_bal_open_trades_l12m,total_bal_open_auto_trades_l12m,months_since_recent_chargedoff180P,total_cl_open_auto_trades_l12m,total_cl_open_pl_trades_l12m,no_of_chargedoff180P,total_cl_of_cc_trades_l12m,no_of_90p_accs_ever,total_bal_of_chargedoff_trades,worst_rating_l12m,current_utilization_of_top_wallet_bankcard,no_of_trades_pl_2w_gl_cd,total_bal_open_trades_l12m_except_wo_cc_od,no_of_deduped_inquiries_l12m,no_of_deduped_inquiries_l6m,no_of_deduped_inquiries_l3m,no_of_deduped_inquiries_l1m,worst_rating_l24m,worst_rating_l6m,worst_rating_l3m,worst_rating_l1m,no_of_60p_accs_l24m,no_of_60p_accs_l12m,no_of_trades_bounced_l36m,no_of_trades_bounced_l12m,no_of_cc_trades_60P_l24m,no_of_30P_cc_l24m,no_of_cc_trades_60P_l12m,no_of_30P_cc_l12m,no_of_cc_lt30p_l12m,no_cc_sanctionedamt_lt30K_l36m,no_pl_sanctionedamt_lt30K_l36m,no_tr_opened_l6m,worst_dpd_l36m,total_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades,no_tr_opened_l12m,no_tr_opened_l3m,per_of_30p_accs_ever,total_payment_amount_bankcard_accs_l3m,max_revolving_monthly_spend_l12m,total_monthly_obligations_l3m,total_balance_bankcard_accs_l3m,no_of_trades_unsec,total_payment_amount_l3m,worst_dpd_l2m,Secured Accounts Count,Unsecured Accounts Count,Secured High Credit Sum,Unsecured High Credit Sum,Secured Amount Overdue Sum,Unsecured Amount Overdue Sum,Secured Balances Sum,Unsecured Balances Sum,Own Accounts count,Other Accounts count,CIBILTUSC3 Score Value,createdate,Current_Salary,available_income,foir_dlq,foir_cibil,foir_trd,foir_mean,New_FOIR,naps_score,rnk
0,8ff8e963-62be-4a29-9a95-727af5659b7a,CAT-C,3258321a-628c-430d-a223-0593c79bd79e,66701.0,26.9,18.0,0.99,PASS,PENDING,0.0,0.0,COMPLETED,COMPLETED,quikr,HiiProp,PRE_APPROVED,9589055084,41f8a882471ed3fbc2657746691e0badecf5f39e8d7f73...,Yes,Self Employed,Health/Hospital,23000,-,service,Residence,Family owned,MARRIED,-,REMOTE,Positive,liquiloans,BANK OF BARODA,2023-03-27,Female,110058,44.0,DELHI,DELHI,Marathi,9cca07c1-86bc-4960-9c7d-e92cb839223f,3bdaa557-9b6a-48aa-8422-ba31c092d1c3,-2.0,-2.0,NOBC,0.0,7.0,-1.0,-1.0,-3.0,184.0,0.0,0.0,0.0,0.0,3.0,178059.0,-1.0,-1.0,-1.0,-3.0,0.0,-3.0,0.0,-1.0,1.0,-2.0,1.0,178059.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,3.0,3.0,0.0,-2.0,0.0,-2.0,1.0,0.0,0.0,0.0,1.0,-1.0,1.0,0.0,0.0,-7.0,-2.0,-2.0,-7.0,3.0,6921.0,0.0,0.0,1.0,0.0,200000.0,0.0,0.0,0.0,178059.0,0.0,1.0,738.0,2022-12-01,48000.00,19097.05,0.5,0.5,0.50,0.50,76.76,744.0,1.0
1,e79c178d-181c-43fb-b86c-413110addb90,CAT-A,a075e629-1f07-4218-a949-820a8ca0ccfd,267903.0,16.8,36.0,0.36,FAIL,COMPLETED,1.0,0.0,COMPLETED,PENDING,niro,LowProp,PRE_APPROVED,9221348191,061c33255a6e18d08fd3ab09734ad30474a853be0c8732...,Yes,Salaried,Purchase,35000,ICMS,-,Residence,Self owned,MARRIED,senior technician,REMOTE,Positive,payu,HDFC BANK,2023-05-11,Male,400605,42.0,MUMBAI,MAHARASHTRA,Kannada,73211792-22e2-4de0-8a9c-337a2a349dd0,d66a3c4f-4f3b-406c-8d07-884c7df72dd0,0.0,0.0,TRANSACTOR,0.0,13.0,-1.0,-1.0,-1.0,140.0,0.0,3.0,0.0,0.0,10.0,1187795.0,1156827.0,-1.0,1189179.0,-1.0,0.0,241778.0,0.0,-1.0,1.0,-2.0,1.0,1156827.0,2.0,2.

In [17]:
final["disbursement_month"] = final["disbursement_date"].astype(str).str[:7]
final.groupby("disbursement_month").agg({"niro_opportunity_id": "count"})

,niro_opportunity_id
disbursement_month,
2021-12,1
2022-01,19
2022-02,54
2022-03,56
2022-04,132
2022-05,217
2022-06,366
2022-07,496
2022-08,805


In [18]:
# amounts validation w.r.t category
final[(final["disbursement_month"] != "2024-01")].groupby("category").agg(
    {"principal_amount": "sum"}
)

,principal_amount
category,
CAT-A,1.040461e+09
CAT-B,4.126023e+09
CAT-C,1.379046e+09
CAT-D,5.597778e+08


In [19]:
# Total amount validation
final["principal_amount"].sum()

7936374494.0

#### Adding repayment data

In [20]:
os.chdir("..")
fileloc = os.getcwd()
fileloc

'/Users/keyurpethad/Library/CloudStorage/GoogleDrive-keyur@niro.money/My Drive/analytics-projects/Portfolio Analysis'

In [21]:
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

In [22]:
perf = pd.read_excel(fileloc+"/inputs/PL Loan Book " + curr_month + ".xlsx", header=2)
perf = perf[~perf['niro_user_id'].isna()]
print("Len of performance file is: "+str(len(perf)))

INFO:snowflake.connector.connection:closed
INFO:snowflake.connector.connection:No async queries seem to be running, deleting session


Len of performance file is: 40069


In [23]:
perf.head()

,niro_user_id,Niro Opportunity ID,Disbursal Partner Name,Platform Partner Name,Lender Loan Id,Disbursement Month,Month Number,Disbursement Year,Phone Numbers,Disbursement Date,Presentation Lag,Gross PF Charged,PF Refund,Final Proc Fees,GST,Pre-EMI,Disbursed Amount,Loan Amount (all inclusive),Loan Amount Rank,Interest Rate,Tenor In Months,Tenor Rank,PF %,Cost of Funds,Cost of Funds (New),Spread,Monthly Emi,Monthly Income (Net Take Home),Income Rank,Income source,Self Employed,Nature of Business,# Years in Employment,Residence Ownership Type,Owned,Purpose of Loan,CIBIL Score,CIBIL Band,CIBIL Score Rank,Risk Band,PL & EL-PL,Disbursement TAT,City,State,Pin Code,Tier,Gender,Male,Female,Age,customer_router,CPV Initiated,Lead Name,Loan Status,Foreclosure Date,Foreclosure Details,NPA Details,NPA setteled Status,Write-off Month,Feb Presentation,Mar Presentation,Apr Presentation,May Presentation,Jun Presentation,Jul Presentation,Aug Presentation,Sep Presentation,Oct Presentation,Nov Presentation,Dec Presentation,Jan'23 Presentation,Feb'23 Presentation,Mar'23 Presentation,Apr'23 Presentation,May'23 Presentation,Jun'23 Presentation,July'23 Presentation,Aug'23 Presentation,Sep'23 Presentation,Oct'23 Presentation,Nov'23 Presentation,Dec'23 Presentation,Jan'24 Presentation,Feb DPD,Mar DPD,Apr DPD,May DPD,Jun DPD,Jul DPD,Aug DPD,Sep DPD,Oct DPD,Nov DPD,Dec DPD,Jan'23 DPD,Feb'23 DPD,Mar'23 DPD,Apr'23 DPD,May'23 DPD,Jun'23 DPD,July'23 DPD,Aug'23 DPD,Sep'23 DPD,Oct'23 DPD,Nov'23 DPD,Dec'23 DPD,Mar DPD Rank,PayU'July'23 DPD,PayU'Aug'23 DPD,PayU'Sep'23 DPD,PayU'Oct'23 DPD,PayU'Nov'23 DPD,PayU'Dec'23 DPD,Sep DPD Rank,Jan - POS,Feb - POS,Mar - POS,Apr - POS,May - POS,Jun - POS,Jul - POS,Aug - POS,Sep - POS,Oct - POS,Nov - POS,Dec - POS,Jan'23 - POS,Feb'23 - POS,Mar'23 - POS,Apr'23 - POS,May'23 - POS,Jun'23 - POS,July'23 - POS,Aug'23 - POS,Sep'23 - POS,Oct'23 - POS,Nov'23 - POS,Dec'23 - POS,Int * Loan Amount (all Incl),CoF * Loan Amount (all Incl),Tenor * Loan Amount (all Incl),PF * Loan Amount (all Incl),Spread * Loan Amount (all Incl),FOIR,Ok/Problem Areas,Repeat Cases,Repeat,DPD 90+ Months,DPD 60+ Months,90+ Amount,Feb - Repaid,Mar - Repaid,Apr - Repaid,May - Repaid,Jun - Repaid,Jul - Repaid,Aug - Repaid,Sep - Repaid,Oct - Repaid,Nov - Repaid,Dec - Repaid,Jan'23 - Repaid,Feb'23 - Repaid,Mar'23 - Repaid,Apr'23 - Repaid,May'23 - Repaid,Jun'23 - Repaid,July'23 - Repaid,Aug'23 - Repaid,Sep'23 - Repaid,Unnamed: 170,Unnamed: 171,Unnamed: 172,Unnamed: 173,Unnamed: 174
0,0a2c0c4e-e0bf-4e50-909e-25ad6272119e,0f100756-11ae-4472-bd29-4fdbd6c37fdb,liquiloans,quikr,499217,2021-12-31,1,2021,9900368067,2021-12-31,Yes,2366,0,2366,426,0.0,50005,52797,2,27.0,12,2,0.044813,17.4,17.4,9.6,5069,26000,3,Salaried,0,'-',1,Rented,0,Others,764,760+,8,CAT-D,EL-PL,< 24 hrs,Bangalore,KARNATAKA,560085,Tier 1,Female,0,1,39,PL_OLD_PREAPPROVED,Yes,VINUTHA HONNE GOWDA,Closed,NaT,NaN,NaN,No,-,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,-,-,-,-,-,-,-,-,-,-,-,NaN,-,-,-,-,-,-,NaN,52797.0,48915.93250,44947.540981,40889.860653,36740.882518,32498.552375,28160.769803,23722.160542,19190.208334,14553.208334,9811.208334,4962.960521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1425519.0,918667.8,633564,2366.0,506851.2,0.44,OK Area,1,Unique,-,-,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-31,1.0
1,e85b616e-e54b-4d67-b929-5f4b612404e2,188ec580-5b20-435b-8b02-b1fd0c439c6b,liquiloans,quikr,500342,2022-01-31,2,2022,8765513171,2022-01-04,No,2366,0,2366,426,0.0,50005,52797,2,27.0,12,2,0.044813,17.4,17.4,9.6,5069,20000,2,Salaried,0,'-',1.5,Owned,1,Marriage,746,740-760,7,CAT-D,EL-PL,< 24 hrs,Lucknow,UTTAR PRADESH,226029,Tier 2,Male,1,0,37,PL_OLD_PREAPPROVED,Yes,AKHILESH KUMAR YADAV,Foreclosed,2022-09-12,NaN,NaN,No,-,Cleared,Cleared,Cleared,Cleared,BOUNCED,BOUNCED

In [24]:
def convert(varOut, VarIn):
    try:
        perf[varOut] = pd.to_numeric(perf[VarIn].str.replace(",","").str.replace("-","").str.strip())
    except:
        perf[varOut] = perf[VarIn]+1-1

convert('Principal','Loan Amount (all inclusive)')
convert('Prin0','Feb - POS')
convert('Prin1','Mar - POS')
convert('Prin2','Apr - POS')
convert('Prin3','May - POS')
convert('Prin4','Jun - POS')
convert('Prin5','Jul - POS')
convert('Prin6','Aug - POS')
convert('Prin7','Sep - POS')
convert('Prin8','Oct - POS')
convert('Prin9','Nov - POS')
convert('Prin10','Dec - POS')
convert('Prin11',"Jan'23 - POS")
convert('Prin12',"Feb'23 - POS")
convert('Prin13',"Mar'23 - POS")
convert('Prin14',"Apr'23 - POS")
convert('Prin15',"May'23 - POS")
convert('Prin16',"Jun'23 - POS")
convert('Prin17',"July'23 - POS")
convert('Prin18',"Aug'23 - POS")
convert('Prin19',"Sep'23 - POS")
convert('Prin20',"Oct'23 - POS")
convert('Prin21',"Nov'23 - POS")
convert('Prin22',"Dec'23 - POS")
# convert('Prin23',"Jan'24 - POS")

perf['DPD0'] = perf['Feb DPD']
perf['DPD1'] = perf['Mar DPD']
perf['DPD2'] = perf['Apr DPD']
perf['DPD3'] = perf['May DPD']
perf['DPD4'] = perf['Jun DPD']
perf['DPD5'] = perf['Jul DPD']
perf['DPD6'] = perf['Aug DPD']
perf['DPD7'] = perf['Sep DPD']
perf['DPD8'] = perf['Oct DPD']
perf['DPD9'] = perf['Nov DPD']
perf['DPD10'] = perf['Dec DPD']
perf['DPD11'] = perf["Jan'23 DPD"]
perf['DPD12'] = perf["Feb'23 DPD"]
perf['DPD13'] = perf["Mar'23 DPD"]
perf['DPD14'] = perf["Apr'23 DPD"]
perf['DPD15'] = perf["May'23 DPD"]
perf['DPD16'] = perf["Jun'23 DPD"]
perf['DPD17'] = perf["July'23 DPD"]
perf['DPD18'] = perf["Aug'23 DPD"]
perf['DPD19'] = perf["Sep'23 DPD"]
perf['DPD20'] = perf["Oct'23 DPD"]
perf['DPD21'] = perf["Nov'23 DPD"]
perf['DPD22'] = perf["Dec'23 DPD"]
# perf['DPD23'] = perf["Jan'24 DPD"]

perf['Presentation0'] = perf['Feb Presentation']
perf['Presentation1'] = perf['Mar Presentation']
perf['Presentation2'] = perf['Apr Presentation']
perf['Presentation3'] = perf['May Presentation']
perf['Presentation4'] = perf['Jun Presentation']
perf['Presentation5'] = perf['Jul Presentation']
perf['Presentation6'] = perf['Aug Presentation']
perf['Presentation7'] = perf['Sep Presentation']
perf['Presentation8'] = perf['Oct Presentation']
perf['Presentation9'] = perf['Nov Presentation']
perf['Presentation10'] = perf['Dec Presentation']
perf['Presentation11'] = perf["Jan'23 Presentation"]
perf['Presentation12'] = perf["Feb'23 Presentation"]
perf['Presentation13'] = perf["Mar'23 Presentation"]
perf['Presentation14'] = perf["Apr'23 Presentation"]
perf['Presentation15'] = perf["May'23 Presentation"]
perf['Presentation16'] = perf["Jun'23 Presentation"]
perf['Presentation17'] = perf["July'23 Presentation"]
perf['Presentation18'] = perf["Aug'23 Presentation"]
perf['Presentation19'] = perf["Sep'23 Presentation"]
perf['Presentation20'] = perf["Oct'23 Presentation"]
perf['Presentation21'] = perf["Nov'23 Presentation"]
perf['Presentation22'] = perf["Dec'23 Presentation"]
perf['Presentation23'] = perf["Jan'24 Presentation"]
# perf['Presentation24'] = perf["Feb'24 Presentation"]

In [25]:
perf.head()

,niro_user_id,Niro Opportunity ID,Disbursal Partner Name,Platform Partner Name,Lender Loan Id,Disbursement Month,Month Number,Disbursement Year,Phone Numbers,Disbursement Date,Presentation Lag,Gross PF Charged,PF Refund,Final Proc Fees,GST,Pre-EMI,Disbursed Amount,Loan Amount (all inclusive),Loan Amount Rank,Interest Rate,Tenor In Months,Tenor Rank,PF %,Cost of Funds,Cost of Funds (New),Spread,Monthly Emi,Monthly Income (Net Take Home),Income Rank,Income source,Self Employed,Nature of Business,# Years in Employment,Residence Ownership Type,Owned,Purpose of Loan,CIBIL Score,CIBIL Band,CIBIL Score Rank,Risk Band,PL & EL-PL,Disbursement TAT,City,State,Pin Code,Tier,Gender,Male,Female,Age,customer_router,CPV Initiated,Lead Name,Loan Status,Foreclosure Date,Foreclosure Details,NPA Details,NPA setteled Status,Write-off Month,Feb Presentation,Mar Presentation,Apr Presentation,May Presentation,Jun Presentation,Jul Presentation,Aug Presentation,Sep Presentation,Oct Presentation,Nov Presentation,Dec Presentation,Jan'23 Presentation,Feb'23 Presentation,Mar'23 Presentation,Apr'23 Presentation,May'23 Presentation,Jun'23 Presentation,July'23 Presentation,Aug'23 Presentation,Sep'23 Presentation,Oct'23 Presentation,Nov'23 Presentation,Dec'23 Presentation,Jan'24 Presentation,Feb DPD,Mar DPD,Apr DPD,May DPD,Jun DPD,Jul DPD,Aug DPD,Sep DPD,Oct DPD,Nov DPD,Dec DPD,Jan'23 DPD,Feb'23 DPD,Mar'23 DPD,Apr'23 DPD,May'23 DPD,Jun'23 DPD,July'23 DPD,Aug'23 DPD,Sep'23 DPD,Oct'23 DPD,Nov'23 DPD,Dec'23 DPD,Mar DPD Rank,PayU'July'23 DPD,PayU'Aug'23 DPD,PayU'Sep'23 DPD,PayU'Oct'23 DPD,PayU'Nov'23 DPD,PayU'Dec'23 DPD,Sep DPD Rank,Jan - POS,Feb - POS,Mar - POS,Apr - POS,May - POS,Jun - POS,Jul - POS,Aug - POS,Sep - POS,Oct - POS,Nov - POS,Dec - POS,Jan'23 - POS,Feb'23 - POS,Mar'23 - POS,Apr'23 - POS,May'23 - POS,Jun'23 - POS,July'23 - POS,Aug'23 - POS,Sep'23 - POS,Oct'23 - POS,Nov'23 - POS,Dec'23 - POS,Int * Loan Amount (all Incl),CoF * Loan Amount (all Incl),Tenor * Loan Amount (all Incl),PF * Loan Amount (all Incl),Spread * Loan Amount (all Incl),FOIR,Ok/Problem Areas,Repeat Cases,Repeat,DPD 90+ Months,DPD 60+ Months,90+ Amount,Feb - Repaid,Mar - Repaid,Apr - Repaid,May - Repaid,Jun - Repaid,Jul - Repaid,Aug - Repaid,Sep - Repaid,Oct - Repaid,Nov - Repaid,Dec - Repaid,Jan'23 - Repaid,Feb'23 - Repaid,Mar'23 - Repaid,Apr'23 - Repaid,May'23 - Repaid,Jun'23 - Repaid,July'23 - Repaid,Aug'23 - Repaid,Sep'23 - Repaid,Unnamed: 170,Unnamed: 171,Unnamed: 172,Unnamed: 173,Unnamed: 174,Principal,Prin0,Prin1,Prin2,Prin3,Prin4,Prin5,Prin6,Prin7,Prin8,Prin9,Prin10,Prin11,Prin12,Prin13,Prin14,Prin15,Prin16,Prin17,Prin18,Prin19,Prin20,Prin21,Prin22,DPD0,DPD1,DPD2,DPD3,DPD4,DPD5,DPD6,DPD7,DPD8,DPD9,DPD10,DPD11,DPD12,DPD13,DPD14,DPD15,DPD16,DPD17,DPD18,DPD19,DPD20,DPD21,DPD22,Presentation0,Presentation1,Presentation2,Presentation3,Presentation4,Presentation5,Presentation6,Presentation7,Presentation8,Presentation9,Presentation10,Presentation11,Presentation12,Presentation13,Presentation14,Presentation15,Presentation16,Presentation17,Presentation18,Presentation19,Presentation20,Presentation21,Presentation22,Presentation23
0,0a2c0c4e-e0bf-4e50-909e-25ad6272119e,0f100756-11ae-4472-bd29-4fdbd6c37fdb,liquiloans,quikr,499217,2021-12-31,1,2021,9900368067,2021-12-31,Yes,2366,0,2366,426,0.0,50005,52797,2,27.0,12,2,0.044813,17.4,17.4,9.6,5069,26000,3,Salaried,0,'-',1,Rented,0,Others,764,760+,8,CAT-D,EL-PL,< 24 hrs,Bangalore,KARNATAKA,560085,Tier 1,Female,0,1,39,PL_OLD_PREAPPROVED,Yes,VINUTHA HONNE GOWDA,Closed,NaT,NaN,NaN,No,-,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,-,-,-,-,-,-,-,-,-,-,-,NaN,-,-,-,-,-,-,NaN,52797.0,48915.93250,44947.540981,40889.860653,36740.882518,32498.552375,28160.769803,23722.160542,19190.208334,14553.208334,9811.208334,4962.960521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1425519.

In [26]:
perf[curr_month + " Presentation bucket"] = np.where(perf[prev_month + " DPD"].isin(['-']) | ((pd.to_numeric(perf[prev_month + " DPD"], errors='coerce') >= -45) & (pd.to_numeric(perf[prev_month + " DPD"], errors='coerce') <= 29)), 'A) Bkt 0',
                                                np.where(perf[prev_month + " DPD"].isin(['1-29+']), 'B) Bkt 1',
                                                np.where(perf[prev_month + " DPD"].isin(['30+']), 'C) Bkt 2',
                                                np.where(perf[prev_month + " DPD"].isin(['60+']), 'D) Bkt 3',
                                                np.where(perf[prev_month + " DPD"].isin(['90+']), 'E) Bkt 3+',
                                                np.where(perf[prev_month + " DPD"].isin(['NPA (Unsettled)', 'NPA']), 'E) Bkt 3+',
                                                np.where(perf[prev_month + " DPD"].isin(['Settled', 'Closed', 'Settled (partial paid)', 'Foreclosed']), 'F) Closed',perf[prev_month + " DPD"])))))))

In [27]:
#check this logic
for x in range(22):
    p=x+1
    print(p)
    perf['Prin'+str(p)] = np.select(condlist=[(perf['Prin'+str(p)]>0).fillna(False),
                                              perf['Presentation'+str(p)].str.upper().str.contains('CLOSE').fillna(False),\
                                              perf['Presentation'+str(p+1)].str.upper().str.contains('CLOSE').fillna(False)],
                                    choicelist=[perf['Prin'+str(p)],0,0],
                                    default = perf['Prin'+str(p-1)])
    

perf['disb_dt'] = pd.to_datetime(perf['Disbursement Date'], format= '%d-%b-%y', errors = 'coerce')
perf['disb_yymm'] = perf['disb_dt'].dt.to_period('M').dt.to_timestamp()
perf['foreclosure_dt'] = pd.to_datetime(perf['Foreclosure Date'], format= '%d-%b-%y', errors = 'coerce')
perf['mth_to_foreclose'] = 12*(perf['foreclosure_dt'].dt.year - perf['disb_yymm'].dt.year)+\
                           (perf['foreclosure_dt'].dt.month - perf['disb_yymm'].dt.month)
##### ------------------ Change here ------------------
perf_enddate = datetime.datetime(2023,12,1)
##### -------------------------------------------------
perf_startdate = datetime.datetime(2022,2,1)

perf['months_since_disb'] = (12*(perf_enddate.year - perf['disb_yymm'].dt.year)+\
                           (perf_enddate.month - perf['disb_yymm'].dt.month)+1).astype(int)
perf['PF'] = pd.to_numeric(perf["PF %"].astype(str).str.replace(" ","").str.replace("%","").str.strip())

# recovery = pd.read_csv(fileloc+'/recovery.csv',low_memory=False)
# recovery['Niro Opportunity ID'] = recovery['Opportunity ID']
# recovery['recovery'] = pd.to_numeric(recovery['POS at time of NPA'].str.replace(",",""),errors='coerce')-\
#                        pd.to_numeric(recovery['Current POS'].str.replace(",",""),errors='coerce')
# recovery = recovery[recovery['recovery']>0]
# recovery = recovery[['Niro Opportunity ID','recovery']]

# perf = perf.merge(recovery,how='left',on='Niro Opportunity ID')
# perf['recovery'].fillna(0,inplace=True)

# perf.to_csv(fileloc+'aa.csv')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


In [28]:
final['ctc'] = pd.to_numeric(final['monthly_take_home_income'].str.replace(",","").str.replace("-",""),errors='coerce')

In [29]:
def cibil_score_bkt(row, var):
    if row[var] <= 736:
        return '<=736'
    elif row[var] <= 745:
        return '737-745'
    elif row[var] <= 749:
        return '746-749'
    elif row[var] <= 757:
        return '750-757'
    elif row[var] <= 760:
        return '758-760'
    elif row[var] <= 766:
        return '761-766'
    elif row[var] <= 772:
        return '767-772'
    elif row[var] <= 776:
        return '773-776'
    elif row[var] <= 784:
        return '777-784'
    else:
        return '785+'

def naps_score_bkt(row, var):
    if row[var] <= 689:
        return '<=689'
    elif row[var] <= 704:
        return '690-704'
    elif row[var] <= 713:
        return '705-713'
    elif row[var] <= 728:
        return '714-728'
    elif row[var] <= 742:
        return '723-742'
    elif row[var] <= 751:
        return '743-751'
    elif row[var] <= 770:
        return '752-770'
    elif row[var] <= 790:
        return '771-790'
    elif row[var] <= 818:
        return '791-818'
    else:
        return '819+'

def napstu_score_bkt(row, var):
    if row[var] <= 694:
        return '<=694'
    elif row[var] <= 705:
        return '695-705'
    elif row[var] <= 714:
        return '706-714'
    elif row[var] <= 727:
        return '715-727'
    elif row[var] <= 743:
        return '728-743'
    elif row[var] <= 759:
        return '744-759'
    elif row[var] <= 780:
        return '760-780'
    elif row[var] <= 805:
        return '781-805'
    elif row[var] <= 847:
        return '806-847'
    else:
        return '848+'


#statename
#BCPMTSTR

final['CIBIL'] = final.apply(cibil_score_bkt,var='CIBILTUSC3 Score Value',axis=1)

final['naps_band'] = final.apply(napstu_score_bkt,var='naps_score',axis=1)

final['no_of_open_cc_trades_bkt'] = np.select(condlist=[final['no_of_open_cc_trades']<=0,
                                             final['no_of_open_cc_trades']<=2,
                                             final['no_of_open_cc_trades']<=4,
                                             final['no_of_open_cc_trades']>4,],choicelist=['No Credit Cards','1-2',
                                                                                        '3-4','5+'],default='No Credit Cards')
final['Years on Bureau_bkt'] = np.select(condlist=[final['months_since_oldest_trade']<=24,
                                             final['months_since_oldest_trade']<=48,
                                             final['months_since_oldest_trade']<=60,
                                             final['months_since_oldest_trade']>60,],choicelist=['1-2 Yrs','2-4 Yrs',
                                                                                        '4-5 Yrs','5+ Yrs'],default='None')
final['no_of_deduped_inquiries_bkt'] = np.select(condlist=[final['no_of_deduped_inquiries_l12m']<=1,
                                             final['no_of_deduped_inquiries_l12m']<=3,
                                             final['no_of_deduped_inquiries_l12m']<=6,
                                             final['no_of_deduped_inquiries_l12m']>6,],choicelist=['<=1','2-3',
                                                                                        '4-6','7+'],default='None')

final['Secured Loan_bkt'] = np.select(condlist=[final['Secured High Credit Sum']<=0,
                                             final['Secured High Credit Sum']<=200000,
                                             final['Secured High Credit Sum']<=750000,
                                             final['Secured High Credit Sum']>750000,],choicelist=['No Secured loan','Secured Loan < 2L',
                                                                                        'Secured Loan < 7.5L','Secured Loan 7.5L+'],
                                  default='No Secured loan')

final['Income_bkt'] = np.select(condlist=[final['ctc']<=15000,
                                            final['ctc']<=20000,
                                            final['ctc']<=25000,
                                             final['ctc']<=30000,
                                             final['ctc']<=35000,
                                            final['ctc']<=40000,
                                            final['ctc']<=50000,
                                            final['ctc']<=75000,
                                             final['ctc']>75000],choicelist=['<=15000','15000-20000','20000-25000','25000-30000',
                                                                                '30000-35000','35000-40000','40000-50000',
                                                                                '50000-75000','75000+'],\
                            default='None')

final['FOIR_Band'] = np.select(condlist=[final['New_FOIR']<=35,
                                             final['New_FOIR']<=50,
                                             final['New_FOIR']<=60,
                                            final['New_FOIR']>60],choicelist=['<=35%','35-50%',
                                                                                        '50-60%','60-70%'],default='None')

#interest_rate
final['Time_on_Bureau_bkt'] = np.select(condlist=[final['months_since_oldest_trade']<=12,
                                                final['months_since_oldest_trade']<=18,
                                             final['months_since_oldest_trade']<=24,
                                             final['months_since_oldest_trade']<=36,
                                            final['months_since_oldest_trade']<=60,
                                            final['months_since_oldest_trade']<=84,
                                            final['months_since_oldest_trade']>84],
                                    choicelist=['<=12Mths','12-18Mths','19-24Mths','2-3 Yrs','3-5 Yrs','5-7 Yrs','7+ Yrs'],
                                    default='None')

final['cust_category'] = np.select(condlist=[((final['no_of_open_cc_trades']>=1) & (final['total_cl_of_cc_trades_l12m']>=50000))&((final['total_cl_open_auto_trades_l12m']>=200000)|(final['total_cl_open_mortgage_trades_l12m']>=1000000)),\
                                          ((final['no_of_open_cc_trades']>=1) & (final['total_cl_of_cc_trades_l12m']>=50000)),
                                          ((final['no_of_open_cc_trades']<=0) & (final['Unsecured High Credit Sum']>=50000))
                                            ],choicelist=['Carded 50K+ With Secured Loans','Carded 50K+',\
                                                          'Unsecured Loan 50K+'],
                                    default='None')

final['cust_hirisk'] = np.select(condlist=[((final['Unsecured High Credit Sum']<=82000) &
                                            (final['no_of_deduped_inquiries_l1m']>=2)&
                                           (final['no_of_trades_pl_2w_gl_cd']>=3))
                                            ],choicelist=[1],default=0)

final['niro_user_id'] = final['user_id'].astype(str).str.strip()


In [30]:
def preprocessor(df):

    try:
        df['CC_INACTIVE'] = np.select(condlist=[(df['bank_card_payment_category']=='INACTIVE')],choicelist=[1],default=0)
        df['CC_NOBC'] = np.select(condlist=[(df['bank_card_payment_category']=='NOBC')],choicelist=[1],default=0)
        df['CC_REVOLVER'] = np.select(condlist=[(df['bank_card_payment_category']=='REVOLVER')],choicelist=[1],default=0)
        df['CC_RVLRPLUS'] = np.select(condlist=[(df['bank_card_payment_category']=='RVLRPLUS')],choicelist=[1],default=0)
        df['CC_TRANSACTOR'] = np.select(condlist=[(df['bank_card_payment_category']=='TRANSACTOR')],choicelist=[1],default=0)
        df['CC_TRANPLUS'] = np.select(condlist=[(df['bank_card_payment_category']=='TRANPLUS')],choicelist=[1],default=0)
    except:
        print("errors")

    df['credithungry'] = np.select(condlist=[((df['UNSECURED_HIGH_CREDIT_SUM']<=75000)&(df['SECURED_HIGH_CREDIT_SUM']<=0)&\
                                            (df['no_of_deduped_inquiries_l6m']>=4))],choicelist=[1],default=0)

    df['no_of_open_cc_trades_rnkXno_of_business_general_trades_rnkXno_of_90p_accs_ever_rnkX3091']=np.select(condlist=[((df['no_of_open_cc_trades']>=2.0)&(df['no_of_open_cc_trades']<=21.0)&(df['no_of_business_general_trades']>=0.0)&(df['no_of_business_general_trades']<=0.0)&(df['no_of_90p_accs_ever']>=1.0)&(df['no_of_90p_accs_ever']<=11.0))],choicelist=[1],default=0)
    df['max_aggregate_bankcard_utilisation_l12m_cc_util_revolving_l1m_6020']=np.select(condlist=[((df['max_aggregate_bankcard_utilisation_l12m']>39.22)&(df['max_aggregate_bankcard_utilisation_l12m']<=80.38)&(df['cc_util_revolving_l1m']>9.21)&(df['cc_util_revolving_l1m']<=538.69))],choicelist=[1],default=0)
    df['no_of_trades_pl_2w_gl_cd_worst_rating_l24m__unsecured_high_credit_sum_42']=np.select(condlist=[((df['no_of_trades_pl_2w_gl_cd']>10)&(df['no_of_trades_pl_2w_gl_cd']<=284)&(df['worst_rating_l24m']>-0.01)&(df['worst_rating_l24m']<=1)&(df['UNSECURED_HIGH_CREDIT_SUM']>520000)&(df['UNSECURED_HIGH_CREDIT_SUM']<=57875849))],choicelist=[1],default=0)
    df['max_aggregate_bankcard_utilisation_l12m_rnkXworst_rating_l3m_rnkXtotal_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades_rnkX150']=np.select(condlist=[((df['max_aggregate_bankcard_utilisation_l12m']>=0.0)&(df['max_aggregate_bankcard_utilisation_l12m']<=34.5)&(df['worst_rating_l3m']>=1.0)&(df['worst_rating_l3m']<=1.0)&(df['total_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades']>=10329.0)&(df['total_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades']<=33080853.0))],choicelist=[1],default=0)
    df['max_aggregate_bankcard_utilisation_l12m_unsecured_high_credit_sum_3292']=np.select(condlist=[((df['max_aggregate_bankcard_utilisation_l12m']>-6.01)&(df['max_aggregate_bankcard_utilisation_l12m']<=-1)&(df['UNSECURED_HIGH_CREDIT_SUM']>574040.33)&(df['UNSECURED_HIGH_CREDIT_SUM']<=1418204.33))],choicelist=[1],default=0)
    df['no_of_trades_total_cl_of_cc_trades_l12m_9968']=np.select(condlist=[((df['no_of_trades']>-0.01)&(df['no_of_trades']<=4)&(df['total_cl_of_cc_trades_l12m']>-5.01)&(df['total_cl_of_cc_trades_l12m']<=-1))],choicelist=[1],default=0)
    df['worst_rating_l3m_no_of_cc_trades_60P_l12m_7369']=np.select(condlist=[((df['worst_rating_l3m']>-0.01)&(df['worst_rating_l3m']<=1)&(df['no_of_cc_trades_60P_l12m']>-1.01)&(df['no_of_cc_trades_60P_l12m']<=0))],choicelist=[1],default=0)
    df['max_aggregate_bankcard_utilisation_l12m_rnkXtotal_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades_rnkXsecured_balances_sum_rnkX447']=np.select(condlist=[((df['max_aggregate_bankcard_utilisation_l12m']>=38.85)&(df['max_aggregate_bankcard_utilisation_l12m']<=1027.37)&(df['total_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades']>=-1.0)&(df['total_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades']<=-1.0)&(df['SECURED_BALANCES_SUM']>=71607.0)&(df['SECURED_BALANCES_SUM']<=495403790.0))],choicelist=[1],default=0)
    df['max_aggregate_bankcard_utilisation_l12m_rnkXtotal_bal_open_auto_trades_l12m_rnkXno_of_deduped_inquiries_l3m_rnkX6945']=np.select(condlist=[((df['max_aggregate_bankcard_utilisation_l12m']>=0.0)&(df['max_aggregate_bankcard_utilisation_l12m']<=38.84)&(df['total_bal_open_auto_trades_l12m']>=-3.0)&(df['total_bal_open_auto_trades_l12m']<=-1.0)&(df['no_of_deduped_inquiries_l3m']>=2.0)&(df['no_of_deduped_inquiries_l3m']<=2.0))],choicelist=[1],default=0)
    df['worst_rating_l24m_no_pl_sanctionedamt_lt30K_l36m_7642']=np.select(condlist=[((df['worst_rating_l24m']>1)&(df['worst_rating_l24m']<=1.5)&(df['no_pl_sanctionedamt_lt30K_l36m']>0)&(df['no_pl_sanctionedamt_lt30K_l36m']<=1))],choicelist=[1],default=0)
    df['worst_rating_l24m_no_of_cc_lt30p_l12m_541']=np.select(condlist=[((df['worst_rating_l24m']>1.5)&(df['worst_rating_l24m']<=9)&(df['no_of_cc_lt30p_l12m']>1)&(df['no_of_cc_lt30p_l12m']<=2))],choicelist=[1],default=0)
    df['no_of_60p_accs_ever_rnkXno_of_deduped_inquiries_l12m_rnkXno_of_30P_cc_l12m_rnkX6968']=np.select(condlist=[((df['no_of_60p_accs_ever']>=0.0)&(df['no_of_60p_accs_ever']<=0.0)&(df['no_of_deduped_inquiries_l12m']>=8.0)&(df['no_of_deduped_inquiries_l12m']<=73.0)&(df['no_of_30P_cc_l12m']>=1.0)&(df['no_of_30P_cc_l12m']<=3.0))],choicelist=[1],default=0)
    df['worst_rating_l3m_no_of_90p_accs_ever_6844']=np.select(condlist=[((df['worst_rating_l3m']>1)&(df['worst_rating_l3m']<=9)&(df['no_of_90p_accs_ever']>0)&(df['no_of_90p_accs_ever']<=16))],choicelist=[1],default=0)
    df['no_of_60p_accs_ever_no_of_30P_cc_l12m_8953']=np.select(condlist=[((df['no_of_60p_accs_ever']>-6.01)&(df['no_of_60p_accs_ever']<=0)&(df['no_of_30P_cc_l12m']>-1)&(df['no_of_30P_cc_l12m']<=0))],choicelist=[1],default=0)
    df['current_utilization_of_top_wallet_bankcard8'] = np.select(condlist=[((df['current_utilization_of_top_wallet_bankcard']>88)&(df['current_utilization_of_top_wallet_bankcard']<=600))],choicelist=[1],default=0)
    df['no_of_deduped_inquiries_by_total_bal_open_trades_l12m'] = np.select(condlist=[df['total_bal_open_trades_l12m']<=0],choicelist=[df['no_of_deduped_inquiries']],default=df['no_of_deduped_inquiries']/df['total_bal_open_trades_l12m'])
    df['no_of_deduped_inquiries_by_total_bal_open_trades_l12m'] = np.select(condlist=[df['no_of_deduped_inquiries_by_total_bal_open_trades_l12m']>0.0156,df['no_of_deduped_inquiries_by_total_bal_open_trades_l12m']<0.00000001],\
                                                        choicelist=[0.0156,0.00000001],default=df['no_of_deduped_inquiries_by_total_bal_open_trades_l12m'])

    df['cc_util_revolving_l1m_unsecured_high_credit_sum_6330']=np.select(condlist=[((df['cc_util_revolving_l1m']>-6.01)&(df['cc_util_revolving_l1m']<=-1)&(df['UNSECURED_HIGH_CREDIT_SUM']>-0.01)&(df['UNSECURED_HIGH_CREDIT_SUM']<=46297.17))],choicelist=[1],default=0)
    df['no_of_60p_accs_ever_rnkXno_of_open_cc_trades_rnkXno_tr_opened_l3m_rnkX1196']=np.select(condlist=[((df['no_of_60p_accs_ever']>=0.0)&(df['no_of_60p_accs_ever']<=0.0)&(df['no_of_open_cc_trades']>=0.0)&(df['no_of_open_cc_trades']<=1.0)&(df['no_tr_opened_l3m']>=2.0)&(df['no_tr_opened_l3m']<=22.0))],choicelist=[1],default=0)
    df['total_bal_open_trades_l12m_no_of_trades_pl_2w_gl_cd_4682']=np.select(condlist=[((df['total_bal_open_trades_l12m']>-5.01)&(df['total_bal_open_trades_l12m']<=28475.83)&(df['no_of_trades_pl_2w_gl_cd']>2)&(df['no_of_trades_pl_2w_gl_cd']<=4))],choicelist=[1],default=0)
    df['cc_util_revolving_l1m_rnkXno_of_deduped_inquiries_l3m_rnkXunsecured_high_credit_sum_rnkX8903']=np.select(condlist=[((df['cc_util_revolving_l1m']>=3.66)&(df['cc_util_revolving_l1m']<=148.68)&(df['no_of_deduped_inquiries_l3m']>=3.0)&(df['no_of_deduped_inquiries_l3m']<=31.0)&(df['UNSECURED_HIGH_CREDIT_SUM']>=123090.0)&(df['UNSECURED_HIGH_CREDIT_SUM']<=565217.0))],choicelist=[1],default=0)
    df['max_aggregate_bankcard_utilisation_l12m_no_of_deduped_inquiries_www']=np.select(condlist=[((df['max_aggregate_bankcard_utilisation_l12m']>-6.01)&(df['max_aggregate_bankcard_utilisation_l12m']<=-2)&(df['no_of_deduped_inquiries']>6)&(df['no_of_deduped_inquiries']<=8))],choicelist=[1],default=0)
    df['months_since_oldest_trade_no_of_deduped_inquiries_l1m']=np.select(condlist=[((df['months_since_oldest_trade']>-5.01)&(df['months_since_oldest_trade']<=24)&(df['no_of_deduped_inquiries_l1m']>-0.01)&(df['no_of_deduped_inquiries_l1m']<=1))],choicelist=[1],default=0)
    df['no_of_90p_accs_ever_no_of_30P_cc_l24m_3957']=np.select(condlist=[((df['no_of_90p_accs_ever']>0)&(df['no_of_90p_accs_ever']<=16)&(df['no_of_30P_cc_l24m']>0)&(df['no_of_30P_cc_l24m']<=27))],choicelist=[1],default=0)
    df['total_bal_open_trades_l12m_worst_rating_l12m']=np.select(condlist=[((df['total_bal_open_trades_l12m']>72909.14)&(df['total_bal_open_trades_l12m']<=145061.71)&(df['worst_rating_l12m']>1)&(df['worst_rating_l12m']<=1.5))],choicelist=[1],default=0)
    df['max_aggregate_bankcard_utilisation_l12m_rnkXno_of_business_general_trades_rnkXworst_rating_l3m_rnkX571']=np.select(condlist=[((df['max_aggregate_bankcard_utilisation_l12m']>=42.59)&(df['max_aggregate_bankcard_utilisation_l12m']<=113.37)&(df['no_of_business_general_trades']>=1.0)&(df['no_of_business_general_trades']<=9.0)&(df['worst_rating_l3m']>=1.5)&(df['worst_rating_l3m']<=9.0))],choicelist=[1],default=0)
    df['no_of_trades_no_of_trades_pl_2w_gl_cd_8980']=np.select(condlist=[((df['no_of_trades']>24)&(df['no_of_trades']<=380)&(df['no_of_trades_pl_2w_gl_cd']>2)&(df['no_of_trades_pl_2w_gl_cd']<=4))],choicelist=[1],default=0)
    df['max_aggregate_bankcard_utilisation_l12m_rnkXtotal_cl_open_pl_trades_l12m_rnkXworst_rating_l3m_rnkX2111']=np.select(condlist=[((df['max_aggregate_bankcard_utilisation_l12m']>=39.1)&(df['max_aggregate_bankcard_utilisation_l12m']<=178.39)&(df['total_cl_open_pl_trades_l12m']>=3600.0)&(df['total_cl_open_pl_trades_l12m']<=200000.0)&(df['worst_rating_l3m']>=1.5)&(df['worst_rating_l3m']<=9.0))],choicelist=[1],default=0)
    df['no_of_60p_accs_ever_no_of_deduped_inquiries_l12m_4']=np.select(condlist=[((df['no_of_60p_accs_ever']>-6.01)&(df['no_of_60p_accs_ever']<=0)&(df['no_of_deduped_inquiries_l12m']>5)&(df['no_of_deduped_inquiries_l12m']<=7))],choicelist=[1],default=0)
    df['worst_rating_l24m_cc_util_revolving_l1m_1387']=np.select(condlist=[((df['worst_rating_l24m']>1.5)&(df['worst_rating_l24m']<=9)&(df['cc_util_revolving_l1m']>-6.01)&(df['cc_util_revolving_l1m']<=-1))],choicelist=[1],default=0)
    df['no_of_trades_total_bal_of_open_pl_l12m_6726']=np.select(condlist=[((df['no_of_trades']>15)&(df['no_of_trades']<=24)&(df['total_bal_of_open_pl_l12m']>247.67)&(df['total_bal_of_open_pl_l12m']<=51449))],choicelist=[1],default=0)
    df['no_of_open_cc_trades_total_bal_open_trades_l12m_5298']=np.select(condlist=[((df['no_of_open_cc_trades']>-1)&(df['no_of_open_cc_trades']<=1)&(df['total_bal_open_trades_l12m']>-5.01)&(df['total_bal_open_trades_l12m']<=28475.83))],choicelist=[1],default=0)
    return df

def getNapsScore(row):
    
    zz = (-1.7249)+(row['no_of_60p_accs_ever_no_of_30P_cc_l12m_8953']*-0.6572)+(row['worst_rating_l6m']*-0.3441)+(row['worst_rating_l24m_cc_util_revolving_l1m_1387']*0.8992)+\
         (row['no_of_open_cc_trades_rnkXno_of_business_general_trades_rnkXno_of_90p_accs_ever_rnkX3091']*1.0623)+(row['current_utilization_of_top_wallet_bankcard8']*0.6604)+(row['no_of_deduped_inquiries_l6m']*0.0361)+\
         (row['max_aggregate_bankcard_utilisation_l12m_cc_util_revolving_l1m_6020']*-0.8515)+(row['no_of_deduped_inquiries_by_total_bal_open_trades_l12m']*31.4963)+(row['no_of_60p_accs_ever_no_of_deduped_inquiries_l12m_4']*-0.452)+\
         (row['no_of_trades_pl_2w_gl_cd_worst_rating_l24m__unsecured_high_credit_sum_42']*-0.7234)+(row['cc_util_revolving_l1m_unsecured_high_credit_sum_6330']*0.3369)+\
         (row['max_aggregate_bankcard_utilisation_l12m_rnkXworst_rating_l3m_rnkXtotal_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades_rnkX150']*2.9448)+(row['no_of_60p_accs_ever_rnkXno_of_open_cc_trades_rnkXno_tr_opened_l3m_rnkX1196']*0.6413)+\
         (row['max_aggregate_bankcard_utilisation_l12m_unsecured_high_credit_sum_3292']*-0.7401)+(row['total_bal_open_trades_l12m_no_of_trades_pl_2w_gl_cd_4682']*-0.5306)+(row['no_of_trades_total_bal_of_open_pl_l12m_6726']*-1.0659)+\
         (row['no_of_trades_total_cl_of_cc_trades_l12m_9968']*0.3131)+(row['cc_util_revolving_l1m_rnkXno_of_deduped_inquiries_l3m_rnkXunsecured_high_credit_sum_rnkX8903']*1.3062)+\
         (row['worst_rating_l3m_no_of_cc_trades_60P_l12m_7369']*-0.6157)+(row['max_aggregate_bankcard_utilisation_l12m_no_of_deduped_inquiries_www']*0.8942)+(row['no_of_open_cc_trades_total_bal_open_trades_l12m_5298']*0.5325)+\
         (row['max_aggregate_bankcard_utilisation_l12m_rnkXtotal_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades_rnkXsecured_balances_sum_rnkX447']*-0.4146)+(row['months_since_oldest_trade_no_of_deduped_inquiries_l1m']*0.2631)+\
         (row['max_aggregate_bankcard_utilisation_l12m_rnkXtotal_bal_open_auto_trades_l12m_rnkXno_of_deduped_inquiries_l3m_rnkX6945']*-1.5711)+(row['no_of_90p_accs_ever_no_of_30P_cc_l24m_3957']*-1.1005)+\
         (row['worst_rating_l24m_no_pl_sanctionedamt_lt30K_l36m_7642']*0.472)+(row['total_bal_open_trades_l12m_worst_rating_l12m']*0.6542)+(\
             row['worst_rating_l24m_no_of_cc_lt30p_l12m_541']*0.8891)+(row['max_aggregate_bankcard_utilisation_l12m_rnkXno_of_business_general_trades_rnkXworst_rating_l3m_rnkX571']*1.6077)+\
             (row['no_of_60p_accs_ever_rnkXno_of_deduped_inquiries_l12m_rnkXno_of_30P_cc_l12m_rnkX6968']*1.6329)+(row['no_of_trades_no_of_trades_pl_2w_gl_cd_8980']*1.9295)+\
             (row['worst_rating_l3m_no_of_90p_accs_ever_6844']*0.9481)+(row['max_aggregate_bankcard_utilisation_l12m_rnkXtotal_cl_open_pl_trades_l12m_rnkXworst_rating_l3m_rnkX2111']*1.0015)

    r = np.exp(zz)/(1+np.exp(zz))
    scr = int(max(350,min(575*(r**(-0.08)),900)))

    return scr

final['SECURED_ACCOUNTS_COUNT']=final['Secured Accounts Count']
final['UNSECURED_ACCOUNTS_COUNT']=final['Unsecured Accounts Count']
final['SECURED_HIGH_CREDIT_SUM']=final['Secured High Credit Sum']
final['UNSECURED_HIGH_CREDIT_SUM']=final['Unsecured High Credit Sum']
final['SECURED_AMOUNT_OVERDUE_SUM']=final['Secured Amount Overdue Sum']
final['UNSECURED_AMOUNT_OVERDUE_SUM']=final['Unsecured Amount Overdue Sum']
final['SECURED_BALANCES_SUM']=final['Secured Balances Sum']
final['UNSECURED_BALANCES_SUM']=final['Unsecured Balances Sum']
              
final = preprocessor(final)
final['naps_new']= final.apply(getNapsScore,axis=1)
final['naps_new_bkt'] = final.apply(naps_score_bkt,var='naps_new',axis=1)

In [31]:
perf_full = pd.DataFrame()
for i in range(len(perf)):
    a = pd.DataFrame()
    disb = str(perf.loc[i,'disb_yymm'].year)+("0"+str(perf.loc[i,'disb_yymm'].month))[-2:]
    for mob in range(perf.loc[i,'months_since_disb']):
        thisdt = perf.loc[i,'disb_yymm']+relativedelta(months=mob)
        month = str(thisdt.year)+("0"+str(thisdt.month))[-2:]
        pos = 0

        ptr = 12*(thisdt.year-perf_startdate.year)+(thisdt.month-perf_startdate.month)

        xPlusDPD = 0
        thirtyPlusDPD = 0
        sixtyPlusDPD = 0
        ninetyPlusDPD = 0
        bounced = 0
        POSactive = 1

        if ptr>=0:
            dpd = perf.loc[i,'DPD'+str(ptr)]
            POS = perf.loc[i,'Prin'+str(ptr)]
            try:
                recovery = perf.loc[i,'recovery']
            except:
                recovery = 0
                
            if POS>0:
                POSactive = 1
            else:
                POSactive = 0
            presStatus = perf.loc[i,'Presentation'+str(ptr)]

            if 'BOUNCE' in str(presStatus).upper():
                bounced=1
            
            if dpd=='90+':
                xPlusDPD = 1
                thirtyPlusDPD = 1
                sixtyPlusDPD = 1
                ninetyPlusDPD = 1
            elif dpd=='Apr90+':
                xPlusDPD = 1
                thirtyPlusDPD = 1
                sixtyPlusDPD = 1
                ninetyPlusDPD = 1
            elif (('NPA' in str(dpd)) or (dpd=='NPA (Unsettled)') or (dpd=='NPA (unsettled)') or (dpd=='Settled')):
                xPlusDPD = 1
                thirtyPlusDPD = 1
                sixtyPlusDPD = 1
                ninetyPlusDPD = 1
            elif dpd=='60+':
                xPlusDPD = 1
                thirtyPlusDPD = 1
                sixtyPlusDPD = 1
                ninetyPlusDPD = 0
            elif dpd=='30+':
                xPlusDPD = 1
                thirtyPlusDPD = 1
                sixtyPlusDPD = 0
                ninetyPlusDPD = 0
            elif dpd=='1-29+':
                xPlusDPD = 1
                thirtyPlusDPD = 0
                sixtyPlusDPD = 0
                ninetyPlusDPD = 0
                
        else:
            cntr = 0
            dpd = '0'
            POS = perf.loc[i,'Principal']
            recovery = 0

        if (ninetyPlusDPD>0):
            POS = POS - recovery
            
        xPlusPOS = xPlusDPD*POS
        thirtyPlusPOS = thirtyPlusDPD*POS
        sixtyPlusPOS = sixtyPlusDPD*POS
        ninetyPlusPOS = ninetyPlusDPD*POS
        bouncedPOS = bounced*POS
        
        tmp = pd.DataFrame({'niro_user_id':[perf.loc[i,'niro_user_id']],'mob':[mob],'disb_dt':[disb],'month':[month],\
                          'POS':[POS],'xPlusDPD':[xPlusDPD],'thirtyPlusDPD':[thirtyPlusDPD],\
                            'sixtyPlusDPD':[sixtyPlusDPD],'ninetyPlusDPD':[ninetyPlusDPD],'xPlusPOS':[xPlusPOS],\
                            'thirtyPlusPOS':[thirtyPlusPOS],'sixtyPlusPOS':[sixtyPlusPOS],\
                            'ninetyPlusPOS':[ninetyPlusPOS],'bounced':[bounced],'bouncedPOS':[bouncedPOS],'POSactive':[POSactive],\
                            'Loan_Amount_perf':[perf.loc[i,'Principal']],'EMI_perf':[perf.loc[i,'Monthly Emi']],'Loan_Tenor_perf':[perf.loc[i,'Tenor In Months']],\
                            'Loan_ROI_perf':[perf.loc[i,'Interest Rate']],'Loan_PF_perf':[perf.loc[i,'PF']],\
                            'City_perf':[perf.loc[i,'City']],'State_perf':[perf.loc[i,'State']],'Age_perf':[perf.loc[i,'Age']],\
                            'Gender_perf':[perf.loc[i,'Gender']], 'pincode_perf':[perf.loc[i,'Pin Code']],\
                            curr_month + " Presentation bucket" : [perf.loc[i,curr_month + " Presentation bucket"]],\
                            curr_month + " Presentation" : [perf.loc[i,curr_month + " Presentation"]]})
        
        a = pd.concat([a,tmp])
    perf_full  = pd.concat([perf_full,a])

In [32]:
perf_full['POS'].fillna(0,inplace=True)
perf_full['xPlusPOS'].fillna(0,inplace=True)
perf_full['thirtyPlusPOS'].fillna(0,inplace=True)
perf_full['sixtyPlusPOS'].fillna(0,inplace=True)
perf_full['ninetyPlusPOS'].fillna(0,inplace=True)

In [33]:
perf_full.head()

,niro_user_id,mob,disb_dt,month,POS,xPlusDPD,thirtyPlusDPD,sixtyPlusDPD,ninetyPlusDPD,xPlusPOS,thirtyPlusPOS,sixtyPlusPOS,ninetyPlusPOS,bounced,bouncedPOS,POSactive,Loan_Amount_perf,EMI_perf,Loan_Tenor_perf,Loan_ROI_perf,Loan_PF_perf,City_perf,State_perf,Age_perf,Gender_perf,pincode_perf,Jan'24 Presentation bucket,Jan'24 Presentation
0,0a2c0c4e-e0bf-4e50-909e-25ad6272119e,0,202112,202112,52797.000000,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,1,52797,5069,12,27.0,0.044813,Bangalore,KARNATAKA,39,Female,560085,A) Bkt 0,Closed
0,0a2c0c4e-e0bf-4e50-909e-25ad6272119e,1,202112,202201,52797.000000,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,1,52797,5069,12,27.0,0.044813,Bangalore,KARNATAKA,39,Female,560085,A) Bkt 0,Closed
0,0a2c0c4e-e0bf-4e50-909e-25ad6272119e,2,202112,202202,48915.932500,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,1,52797,5069,12,27.0,0.044813,Bangalore,KARNATAKA,39,Female,560085,A) Bkt 0,Closed
0,0a2c0c4e-e0bf-4e50-909e-25ad6272119e,3,202112,202203,44947.540981,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,1,52797,5069,12,27.0,0.044813,Bangalore,KARNATAKA,39,Female,560085,A) Bkt 0,Closed
0,0a2c0c4e-e0bf-4e50-909e-25ad6272119e,4,202112,202204,40889.860653,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,1,52797,5069,12,27.0,0.044813,Bangalore,KARNATAKA,39,Female,560085,A) Bkt 0,Closed


In [34]:
perf_data = perf_full.drop_duplicates(subset = ['niro_user_id'])[['niro_user_id', 'Loan_Amount_perf', 'EMI_perf', 'Loan_Tenor_perf', 'Loan_ROI_perf', 
                                                      'Loan_PF_perf', 'City_perf', 'State_perf', 'Age_perf', 'Gender_perf', 'pincode_perf', curr_month + " Presentation bucket", curr_month + " Presentation"]]

#### Event Data creation

In [35]:
master_data = final.merge(perf_full.loc[(perf_full.mob == 3), ['niro_user_id', 'xPlusDPD', 'thirtyPlusDPD', 'sixtyPlusDPD', 'ninetyPlusDPD', 'xPlusPOS', 'thirtyPlusPOS', 'sixtyPlusPOS', 'ninetyPlusPOS', 'bounced', 'bouncedPOS']].rename(columns = {'xPlusDPD': 'xP3M', 'thirtyPlusDPD': '30P3M','sixtyPlusDPD':'60P3M','ninetyPlusDPD': '90P3M', 'xPlusPOS': 'POS xP3M', 'thirtyPlusPOS': 'POS 30P3M', 'sixtyPlusPOS': 'POS 60P3M', 'ninetyPlusPOS': 'POS 90P3M', 'bounced': 'bounced 3M', 'bouncedPOS': 'bouncedPOS 3M'}), on = 'niro_user_id', how = 'left') \
.merge(perf_full.loc[(perf_full.mob == 6), ['niro_user_id', 'xPlusDPD', 'thirtyPlusDPD', 'sixtyPlusDPD', 'ninetyPlusDPD', 'xPlusPOS', 'thirtyPlusPOS', 'sixtyPlusPOS', 'ninetyPlusPOS', 'bounced', 'bouncedPOS']].rename(columns = {'xPlusDPD': 'xP6M', 'thirtyPlusDPD': '30P6M','sixtyPlusDPD':'60P6M','ninetyPlusDPD': '90P6M', 'xPlusPOS': 'POS xP6M', 'thirtyPlusPOS': 'POS 30P6M', 'sixtyPlusPOS': 'POS 60P6M', 'ninetyPlusPOS': 'POS 90P6M', 'bounced': 'bounced 6M', 'bouncedPOS': 'bouncedPOS 6M'}), on = 'niro_user_id', how = 'left') \
.merge(perf_full.loc[(perf_full.mob == 12), ['niro_user_id', 'xPlusDPD', 'thirtyPlusDPD', 'sixtyPlusDPD', 'ninetyPlusDPD', 'xPlusPOS', 'thirtyPlusPOS', 'sixtyPlusPOS', 'ninetyPlusPOS', 'bounced', 'bouncedPOS']].rename(columns = {'xPlusDPD': 'xP12M', 'thirtyPlusDPD': '30P12M','sixtyPlusDPD':'60P12M','ninetyPlusDPD': '90P12M', 'xPlusPOS': 'POS xP12M', 'thirtyPlusPOS': 'POS 30P12M', 'sixtyPlusPOS': 'POS 60P12M', 'ninetyPlusPOS': 'POS 90P12M', 'bounced': 'bounced 12M', 'bouncedPOS': 'bouncedPOS 12M'}), on = 'niro_user_id', how = 'left') \
.merge(perf_data, on = 'niro_user_id', how = 'left')

In [36]:
print('Shape before dropping: ', master_data.shape)
master_data.drop_duplicates(subset=['niro_user_id'], keep='first',inplace=True)
print('Shape after dropping: ', master_data.shape)

Shape before dropping:  (46294, 225)
Shape after dropping:  (44316, 225)


#### Bucketing logic

In [37]:
master_data['Age_perf_bkt'] = np.where(master_data['Age_perf'] <= 22, 'A) 21-22',
                                np.where(master_data['Age_perf'] <= 25, 'B) 23-25',
                                    np.where(master_data['Age_perf'] <= 30, 'C) 26-30',
                                        np.where(master_data['Age_perf'] <= 35, 'D) 31-35',
                                            np.where(master_data['Age_perf'] <= 40, 'E) 36-40',
                                                np.where(master_data['Age_perf'] <= 45, 'F) 41-45',
                                                    np.where(master_data['Age_perf'] <= 50, 'G) 46-50',
                                                        np.where(master_data['Age_perf'] > 50, 'H) 50+',np.nan))))))))

#### Pincode mapping

In [38]:
master_data['pincode_mismatch'] = np.where(master_data['pincode_perf'].isna() | master_data['cibil_pincode'].isna(),np.nan,
                                        np.where(master_data['pincode_perf'] != master_data['cibil_pincode'],1,0))

In [39]:
master_data['pincode_perf'] = master_data['pincode_perf'].astype('Int64')
master_data['cibil_pincode'] = master_data['cibil_pincode'].astype('Int64')

In [40]:
df = pd.read_csv('PL_pincode_mapping.csv')
df.head()

,CURRENT_LOCATION_CLASSIFICATION,DISTRICT,EFFECTIVE_START_DATE,PERMHOUSE_LOCATION_CLASSIFICATION,PINCODE,STATE,TALUK
0,GREEN,BALESWAR,02-06-2023,GREEN,756029,ODISHA,BASTA
1,GREEN,KHORDA,02-06-2023,GREEN,751016,ODISHA,BHUBANESWAR
2,RED,KULLU,02-06-2023,GREEN,175136,HIMACHAL PRADESH,KULLU(T)
3,GREEN,CUTTACK,02-06-2023,GREEN,754022,ODISHA,TANGI CHOUDWAR
4,RED,BOKARO,02-06-2023,GREEN,827012,JHARKHAND,CHAS


In [41]:
df_ll = pd.read_excel('All-active-Pincodes-Niro.xlsx', sheet_name='LL')
df_payu = pd.read_excel('All-active-Pincodes-Niro.xlsx', sheet_name='Payu')
df_muthoot = pd.read_excel('All-active-Pincodes-Niro.xlsx', sheet_name='Muthoot')
df_piramal = pd.read_excel('All-active-Pincodes-Niro.xlsx', sheet_name='Piramal')

In [42]:
df_ll.head()

,Pincode,City,State,pin_service_ll
0,110001,Central Delhi,DELHI,1
1,110002,Central Delhi,DELHI,1
2,110003,Central Delhi,DELHI,1
3,110004,Central Delhi,DELHI,1
4,110005,Central Delhi,DELHI,1


In [43]:
master_data = master_data.merge(df[['PINCODE', 'CURRENT_LOCATION_CLASSIFICATION']], left_on = 'pincode_perf', right_on = 'PINCODE', how = 'left').rename(columns = {'CURRENT_LOCATION_CLASSIFICATION':'pincode_perf_classification'}).drop(columns = 'PINCODE') \
                .merge(df_ll[['Pincode', 'pin_service_ll']], left_on = 'pincode_perf', right_on = 'Pincode', how = 'left').drop(columns = 'Pincode') \
                .merge(df_payu[['Pincode', 'pin_service_payu']], left_on = 'pincode_perf', right_on = 'Pincode', how = 'left').drop(columns = 'Pincode') \
                .merge(df_muthoot[['Pincode', 'pin_service_muthoot']], left_on = 'pincode_perf', right_on = 'Pincode', how = 'left').drop(columns = 'Pincode') \
                .merge(df_piramal[['Pincode', 'pin_service_piramal']], left_on = 'pincode_perf', right_on = 'Pincode', how = 'left').drop(columns = 'Pincode')

In [44]:
master_data.head()

,niro_opportunity_id,category,user_id,principal_amount,interest_rate,tenure,selfie_liveness_score,selfie_liveness_result,kyc_status,pan_retry_count,nach_retry_count,remote_cpv_status,ckyc_status,supply,propensity,offer_type,phone_number,hashed_phone,customer_availability,income_source,purpose_of_loan,monthly_take_home_income,employer_name,nature_of_business,cpv_need_to_initiate_at,residence_ownership_type,marital_status,designation,cpv_type,agency_status,demand,bank_name,disbursement_date,cibil_gender,cibil_pincode,Calculated_Age,cibil_city,cibil_state,native_language,tli_id,pii_id,max_aggregate_bankcard_utilisation_l12m,cc_util_revolving_l1m,bank_card_payment_category,no_of_60p_accs_ever,no_of_deduped_inquiries,total_cl_open_mortgage_trades_l12m,total_balance_open_mortgage_trades_l12m,total_bal_of_open_pl_l12m,months_since_oldest_trade,no_of_mortgage_trades,no_of_open_cc_trades,no_of_business_general_trades,no_of_30p_accs_ever,no_of_trades,total_bal_open_trades_l12m,total_bal_open_auto_trades_l12m,months_since_recent_chargedoff180P,total_cl_open_auto_trades_l12m,total_cl_open_pl_trades_l12m,no_of_chargedoff180P,total_cl_of_cc_trades_l12m,no_of_90p_accs_ever,total_bal_of_chargedoff_trades,worst_rating_l12m,current_utilization_of_top_wallet_bankcard,no_of_trades_pl_2w_gl_cd,total_bal_open_trades_l12m_except_wo_cc_od,no_of_deduped_inquiries_l12m,no_of_deduped_inquiries_l6m,no_of_deduped_inquiries_l3m,no_of_deduped_inquiries_l1m,worst_rating_l24m,worst_rating_l6m,worst_rating_l3m,worst_rating_l1m,no_of_60p_accs_l24m,no_of_60p_accs_l12m,no_of_trades_bounced_l36m,no_of_trades_bounced_l12m,no_of_cc_trades_60P_l24m,no_of_30P_cc_l24m,no_of_cc_trades_60P_l12m,no_of_30P_cc_l12m,no_of_cc_lt30p_l12m,no_cc_sanctionedamt_lt30K_l36m,no_pl_sanctionedamt_lt30K_l36m,no_tr_opened_l6m,worst_dpd_l36m,total_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades,no_tr_opened_l12m,no_tr_opened_l3m,per_of_30p_accs_ever,total_payment_amount_bankcard_accs_l3m,max_revolving_monthly_spend_l12m,total_monthly_obligations_l3m,total_balance_bankcard_accs_l3m,no_of_trades_unsec,total_payment_amount_l3m,worst_dpd_l2m,Secured Accounts Count,Unsecured Accounts Count,Secured High Credit Sum,Unsecured High Credit Sum,Secured Amount Overdue Sum,Unsecured Amount Overdue Sum,Secured Balances Sum,Unsecured Balances Sum,Own Accounts count,Other Accounts count,CIBILTUSC3 Score Value,createdate,Current_Salary,available_income,foir_dlq,foir_cibil,foir_trd,foir_mean,New_FOIR,naps_score,rnk,disbursement_month,ctc,CIBIL,naps_band,no_of_open_cc_trades_bkt,Years on Bureau_bkt,no_of_deduped_inquiries_bkt,Secured Loan_bkt,Income_bkt,FOIR_Band,Time_on_Bureau_bkt,cust_category,cust_hirisk,niro_user_id,SECURED_ACCOUNTS_COUNT,UNSECURED_ACCOUNTS_COUNT,SECURED_HIGH_CREDIT_SUM,UNSECURED_HIGH_CREDIT_SUM,SECURED_AMOUNT_OVERDUE_SUM,UNSECURED_AMOUNT_OVERDUE_SUM,SECURED_BALANCES_SUM,UNSECURED_BALANCES_SUM,CC_INACTIVE,CC_NOBC,CC_REVOLVER,CC_RVLRPLUS,CC_TRANSACTOR,CC_TRANPLUS,credithungry,no_of_open_cc_trades_rnkXno_of_business_general_trades_rnkXno_of_90p_accs_ever_rnkX3091,max_aggregate_bankcard_utilisation_l12m_cc_util_revolving_l1m_6020,no_of_trades_pl_2w_gl_cd_worst_rating_l24m__unsecured_high_credit_sum_42,max_aggregate_bankcard_utilisation_l12m_rnkXworst_rating_l3m_rnkXtotal_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades_rnkX150,max_aggregate_bankcard_utilisation_l12m_unsecured_high_credit_sum_3292,no_of_trades_total_cl_of_cc_trades_l12m_9968,worst_rating_l3m_no_of_cc_trades_60P_l12m_7369,max_aggregate_bankcard_utilisation_l12m_rnkXtotal_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades_rnkXsecured_balances_sum_rnkX447,max_aggregate_bankcard_utilisation_l12m_rnkXtotal_bal_open_auto_trades_l12m_rnkXno_of_deduped_inquiries_l3m_rnkX6945,worst_rating_l24m_no_pl_sanctionedamt_lt30K_l36m_7642,worst_rating_l24m_no_of_cc_lt30p_l12m_541,no_of_60p_accs_ever_rnkXno_of_deduped_inquiries_l12m_rnkXno_of_30P_cc_l12m_rnkX6968,worst_rating_l3m_no_of_90p_accs_ever_6844,no_of_60p_acc

In [45]:
master_data[curr_month + "_Bounced_flag"] = np.where(master_data[curr_month + " Presentation"].isin(['Bounced', 'Bounced (DPD)', 'NPA', 'NPA (unsettled)']),1,0)
master_data[curr_month + "_T-Bounced_flag"] = np.where(master_data[curr_month + " Presentation"].isin(['T-Bounce']),1,0)

In [46]:
master_data.to_excel(os.getcwd() + '/Analysis/master_data_Jan_2102222.xlsx', index=False)